## Entrenamiento para usar python 

In [1]:
def convertToBase13(num):
    if num == 0:
        return "0"

    # Digits for base 13
    base13_digits = "0123456789ABC"  
    digits = ""
    positive = abs(num)
    
    while positive > 0:
        # Append digits/letters
        digits += base13_digits[positive % 13]  
        positive = positive // 13

    # Reverse the string at the end
    reversed_digits = digits[::-1]  
    if num < 0:
        return "-" + reversed_digits
    else:
        return reversed_digits

In [5]:
num = int(input("Escriba un numero"))
convertToBase13(num)

'A'

In [2]:
def func(x):
    if x == 1:
        def rv():
            print("X is equal to 1")
    else:
        def rv():
            print("X is not 1")

    return rv

In [5]:
new_func = func(2)
new_func

<function __main__.func.<locals>.rv()>

In [6]:
import inspect
from queue import Queue

print(inspect.getsource(Queue))

class Queue:
    '''Create a queue object with a given maximum size.

    If maxsize is <= 0, the queue size is infinite.
    '''

    def __init__(self, maxsize=0):
        self.maxsize = maxsize
        self._init(maxsize)

        # mutex must be held whenever the queue is mutating.  All methods
        # that acquire mutex must release it before returning.  mutex
        # is shared between the three conditions, so acquiring and
        # releasing the conditions also acquires and releases mutex.
        self.mutex = threading.Lock()

        # Notify not_empty whenever an item is added to the queue; a
        # thread waiting to get is notified then.
        self.not_empty = threading.Condition(self.mutex)

        # Notify not_full whenever an item is removed from the queue;
        # a thread waiting to put is notified then.
        self.not_full = threading.Condition(self.mutex)

        # Notify all_tasks_done whenever the number of unfinished tasks
        # drops to zero; th

# Validador de Agendamiento de Reuniones (Calendar Booking System)

## 📝 Descripción:
Construirás un sistema de agendamiento de reuniones que:

    - Permita a usuarios crear reuniones con fecha, hora de inicio y duración.

    - Valide que no haya solapamientos entre reuniones existentes.

    - Permita consultar las reuniones existentes de un día.

    - Permita cancelar reuniones.

## 🎯 Requisitos funcionales:
    
    - Crear reunión (schedule_meeting)

    - Parámetros: start_time, duration, description

    - Valida que no se cruce con ninguna otra existente.

    - Listar reuniones (list_meetings(date))

    - Devuelve lista ordenada por hora.

    - Cancelar reunión (cancel_meeting(id))

## 💡 Consideraciones técnicas:

    - Usa datetime para manejar fechas y horas.

    - Las reuniones duran una cantidad de minutos (duration).

    - Valida que una reunión no se cruce en el tiempo con otra.

In [1]:
from datetime import datetime, timedelta
from typing import List
import uuid

class Meeting:
    def __init__(self, start_time: datetime, duration: int, description: str):
        self.id = str(uuid.uuid4())
        self.start_time = start_time
        self.end_time = start_time + timedelta(minutes=duration)
        self.duration = duration
        self.description = description

    def __str__(self):
        return f"{self.description} ({self.start_time.strftime('%H:%M')} - {self.end_time.strftime('%H:%M')})"

class MeetingScheduler:
    def __init__(self):
        self.meetings: List[Meeting] = []

    def schedule_meeting(self, start: str, duration: int, description: str):
        start_dt = datetime.strptime(start, "%Y-%m-%d %H:%M")
        new_meeting = Meeting(start_dt, duration, description)

        for meeting in self.meetings:
            if self._overlaps(meeting, new_meeting):
                raise ValueError("La nueva reunión se cruza con una existente.")

        self.meetings.append(new_meeting)
        print(f"Reunión programada: {new_meeting}")

    def _overlaps(self, m1: Meeting, m2: Meeting) -> bool:
        return m1.start_time < m2.end_time and m2.start_time < m1.end_time

    def list_meetings(self, date: str):
        date_dt = datetime.strptime(date, "%Y-%m-%d").date()
        filtered = [m for m in self.meetings if m.start_time.date() == date_dt]
        for m in sorted(filtered, key=lambda x: x.start_time):
            print(f"[{m.id[:6]}] {m}")

    def cancel_meeting(self, meeting_id: str):
        before = len(self.meetings)
        self.meetings = [m for m in self.meetings if m.id != meeting_id]
        after = len(self.meetings)
        if before == after:
            print("ID no encontrado.")
        else:
            print("Reunión cancelada.")

In [3]:
calendar = MeetingScheduler()

calendar.schedule_meeting("2025-07-17 09:00", 60, "Reunión con cliente")
calendar.schedule_meeting("2025-07-17 10:30", 30, "Planificación interna")

Reunión programada: Reunión con cliente (09:00 - 10:00)
Reunión programada: Planificación interna (10:30 - 11:00)


In [5]:
calendar.list_meetings("2025-07-17")
# → Reunión con cliente (9:00 - 10:00), Planificación interna (10:30 - 11:00)

[57eb5e] Reunión con cliente (09:00 - 10:00)
[90a3e3] Planificación interna (10:30 - 11:00)


In [6]:
calendar.schedule_meeting("2025-07-17 09:30", 30, "Solapada")  
# → Error: se cruza con otra

ValueError: La nueva reunión se cruza con una existente.

# Diseña un sistema de detección de tasa de peticiones (Rate Limiter)

## 💼 Descripción:
Vas a construir un Rate Limiter, es decir, un componente que impide que un usuario (o IP, o servicio) haga demasiadas peticiones en un corto periodo de tiempo.

    - Este patrón es usado en producción por:

    - APIs públicas (OpenAI, Twitter, GitHub)

    - Servicios que protegen de abuso (login, spam, etc.)

## 🎯 Requisitos:
Solo se deben permitir N peticiones por usuario por ventana de tiempo T (por ejemplo, 5 peticiones cada 10 segundos).

    - Rechaza peticiones que excedan ese límite.

    - Opcional: Implementa variantes como sliding window o leaky bucket si quieres ir más lejos.

In [1]:
from collections import deque
from typing import Dict

class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.user_requests: Dict[str, deque] = {}

    def allow_request(self, user_id: str, current_time: int) -> bool:
        """
        Registra y decide si la petición puede hacerse en el timestamp actual.
        """
        if user_id not in self.user_requests:
            self.user_requests[user_id] = deque()

        requests = self.user_requests[user_id]

        # Elimina peticiones fuera de la ventana de tiempo
        while requests and current_time - requests[0] >= self.window_seconds:
            requests.popleft()

        if len(requests) < self.max_requests:
            requests.append(current_time)
            return True
        return False

In [4]:
limiter = RateLimiter(max_requests=3, window_seconds=10)

#print(limiter.allow_request("user1", 1))   # True
#print(limiter.allow_request("user1", 2))   # True
#print(limiter.allow_request("user1", 3))   # True
#print(limiter.allow_request("user1", 4))   # False (excede el límite)
print(limiter.allow_request("user1", 12))  # True (ventana reseteada)


True


## Sistema de Clasificación de Productos con Búsqueda y Filtros Eficientes

### 📝 Descripción:
Vas a construir un motor de búsqueda y clasificación de productos (como los usados en Alibaba, JD.com, Pinduoduo), que:

1. Permite registrar productos con:

    - id, nombre, precio, categoría, rating, stock

2. Permite realizar búsquedas filtradas por:

    - Categoría

    - Precio mínimo/máximo

    - Rating mínimo

    - Palabras clave en el nombre

3. Permite ordenar los resultados por:

    - Precio (asc o desc)

    - Rating

    - Relevancia (match en nombre)

In [1]:
from typing import List, Optional

class Product:
    def __init__(self, pid: int, name: str, price: float, category: str, rating: float, stock: int):
        self.pid = pid
        self.name = name
        self.price = price
        self.category = category
        self.rating = rating
        self.stock = stock

    def __repr__(self):
        return f"{self.name} - ${self.price} - ⭐{self.rating}"

class ProductSearchEngine:
    def __init__(self):
        self.products: List[Product] = []

    def add_product(self, pid: int, name: str, price: float, category: str, rating: float, stock: int):
        self.products.append(Product(pid, name, price, category, rating, stock))

    def search(self,
               category: Optional[str] = None,
               min_price: Optional[float] = None,
               max_price: Optional[float] = None,
               min_rating: Optional[float] = None,
               keyword: Optional[str] = None,
               sort_by: Optional[str] = None,
               ascending: bool = False) -> List[Product]:

        results = self.products

        if category:
            results = [p for p in results if p.category.lower() == category.lower()]
        if min_price is not None:
            results = [p for p in results if p.price >= min_price]
        if max_price is not None:
            results = [p for p in results if p.price <= max_price]
        if min_rating is not None:
            results = [p for p in results if p.rating >= min_rating]
        if keyword:
            results = [p for p in results if keyword.lower() in p.name.lower()]

        if sort_by == "price":
            results.sort(key=lambda p: p.price, reverse=not ascending)
        elif sort_by == "rating":
            results.sort(key=lambda p: p.rating, reverse=not ascending)
        elif sort_by == "name":
            results.sort(key=lambda p: p.name)

        return results

In [2]:
store = ProductSearchEngine()

store.add_product(1, "Smartphone Xiaomi Redmi", 799, "Electrónica", 4.5, 10)
store.add_product(2, "Xiaomi Band 7", 199, "Electrónica", 4.8, 5)
store.add_product(3, "Lenovo Laptop IdeaPad", 1200, "Computación", 4.3, 7)

result = store.search(category="Electrónica", min_price=100, max_price=800, sort_by="rating")

# → Devuelve productos 2 y 1 ordenados por rating descendente

In [3]:
result

[Xiaomi Band 7 - $199 - ⭐4.8, Smartphone Xiaomi Redmi - $799 - ⭐4.5]

## Sistema de Reservas de Salas con Conflicto Mínimo (Smart Room Allocator)

### 📝 Descripción:
Vas a construir un sistema que administre reservas de salas en una empresa. Pero con una diferencia clave:
Si no hay sala disponible en el horario solicitado, el sistema debe encontrar la sala con el menor conflicto posible (tiempo libre más cercano) o sugerir una alternativa.

Este ejercicio está inspirado en los sistemas de calendarización de Yandex o Tinkoff.

### 🎯 Requisitos funcionales:
1. Puedes registrar salas (add_room(nombre)).

2. Puedes hacer una reserva (book_room(nombre_sala, fecha, hora_inicio, duración_minutos)).

3. Si no hay disponibilidad exacta:

    - Sugerir la sala con el menor tiempo de conflicto.

    - O decir que no hay forma de agendar hoy.

4. Puedes listar reservas por sala y fecha.

In [4]:
from datetime import datetime, timedelta
from typing import List, Dict

class Reservation:
    def __init__(self, start: datetime, duration: int):
        self.start = start
        self.end = start + timedelta(minutes=duration)

    def overlaps(self, other: 'Reservation') -> bool:
        return self.start < other.end and other.start < self.end

    def __str__(self):
        return f"{self.start.strftime('%H:%M')} - {self.end.strftime('%H:%M')}"

class RoomScheduler:
    def __init__(self):
        self.rooms: Dict[str, List[Reservation]] = {}

    def add_room(self, room_name: str):
        if room_name in self.rooms:
            print("La sala ya existe.")
        else:
            self.rooms[room_name] = []

    def book_room(self, room_name: str, date: str, time: str, duration: int):
        if room_name not in self.rooms:
            print(f"Sala {room_name} no registrada.")
            return

        start = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
        new_reservation = Reservation(start, duration)

        for res in self.rooms[room_name]:
            if res.overlaps(new_reservation):
                print(f"⚠️ Conflicto en {room_name} a las {res}")
                self.suggest_alternative(room_name, date, duration)
                return

        self.rooms[room_name].append(new_reservation)
        print(f"✅ Reserva confirmada en {room_name} de {new_reservation}")

    def suggest_alternative(self, room_name: str, date: str, duration: int):
        print("🔍 Buscando alternativa...")
        for alt_room, reservations in self.rooms.items():
            if alt_room == room_name:
                continue
            start_time = datetime.strptime(f"{date} 08:00", "%Y-%m-%d %H:%M")
            end_time = datetime.strptime(f"{date} 18:00", "%Y-%m-%d %H:%M")

            while start_time + timedelta(minutes=duration) <= end_time:
                trial = Reservation(start_time, duration)
                if all(not r.overlaps(trial) for r in reservations):
                    print(f"💡 Alternativa: reservar {alt_room} a las {start_time.strftime('%H:%M')}")
                    return
                start_time += timedelta(minutes=15)
        print("❌ No hay alternativa disponible hoy.")

    def list_reservations(self, room_name: str, date: str):
        print(f"📋 Reservas para {room_name} en {date}:")
        for r in sorted(self.rooms.get(room_name, []), key=lambda x: x.start):
            if r.start.strftime("%Y-%m-%d") == date:
                print(f" - {r}")

In [5]:
scheduler = RoomScheduler()
scheduler.add_room("Sala A")
scheduler.add_room("Sala B")

scheduler.book_room("Sala A", "2025-07-17", "09:00", 60)
scheduler.book_room("Sala A", "2025-07-17", "10:00", 30)

# Intento hacer reserva que se cruza
scheduler.book_room("Sala A", "2025-07-17", "09:30", 30)
# → Sugerencia: usar Sala B o mover a 10:30

scheduler.list_reservations("Sala A", "2025-07-17")

✅ Reserva confirmada en Sala A de 09:00 - 10:00
✅ Reserva confirmada en Sala A de 10:00 - 10:30
⚠️ Conflicto en Sala A a las 09:00 - 10:00
🔍 Buscando alternativa...
💡 Alternativa: reservar Sala B a las 08:00
📋 Reservas para Sala A en 2025-07-17:
 - 09:00 - 10:00
 - 10:00 - 10:30


## Simulador de Tienda de Café
📝 Descripción:
Vas a crear un programa que simule una tienda de café. El sistema debe:

Mostrar un menú con productos y precios.

Permitir al usuario seleccionar productos y cantidades.

Calcular el total de la compra.

Permitir pagar con diferentes billetes/monedas y dar cambio.

Mostrar un recibo final.


In [1]:
class CoffeeShop:
    def __init__(self):
        self.menu = {
            1: ("Espresso", 3),
            2: ("Cappuccino", 4),
            3: ("Latte", 5)
        }
        self.order = []

    def show_menu(self):
        print("Bienvenido a Café Python ☕")
        for key, (name, price) in self.menu.items():
            print(f"{key}. {name} - ${price}")

    def take_order(self):
        while True:
            choice = int(input("Seleccione un producto (1-3) o 0 para finalizar: "))
            if choice == 0:
                break
            if choice not in self.menu:
                print("Opción inválida.")
                continue
            qty = int(input("Cantidad: "))
            self.order.append((choice, qty))

    def calculate_total(self):
        return sum(self.menu[choice][1] * qty for choice, qty in self.order)

    def process_payment(self, total):
        paid = int(input(f"Total a pagar: ${total}\nIngrese pago: $"))
        if paid < total:
            print("Pago insuficiente.")
            return self.process_payment(total)
        print(f"Cambio: ${paid - total}")

    def print_receipt(self):
        print("\n--- Recibo ---")
        for choice, qty in self.order:
            name, price = self.menu[choice]
            print(f"{name} x{qty} = ${price * qty}")
        print("--------------")

    def run(self):
        self.show_menu()
        self.take_order()
        total = self.calculate_total()
        self.process_payment(total)
        self.print_receipt()
        print("Gracias por su compra.")

# Ejecutar
shop = CoffeeShop()
shop.run()

Bienvenido a Café Python ☕
1. Espresso - $3
2. Cappuccino - $4
3. Latte - $5


ValueError: invalid literal for int() with base 10: ''

Ejercicios básicos:
- Función para saludar: Crea una función que reciba un nombre como parámetro y devuelva un saludo personalizado.
- Función para calcular el área de un círculo: Recibe el radio como parámetro y devuelve el área.
- Función para verificar si un número es par: Recibe un número como parámetro y devuelve True si es par, False en caso contrario.
- Función para convertir grados Celsius a Fahrenheit: Recibe la temperatura en Celsius y devuelve la temperatura en Fahrenheit.
- Función para encontrar el valor máximo de una lista: Recibe una lista de números como parámetro y devuelve el valor máximo. 

Ejercicios con más lógica:
- Función para calcular el factorial de un número: Recibe un número entero positivo y devuelve su factorial. 
- Función para verificar si una cadena es un palíndromo: Recibe una cadena como parámetro y devuelve True si es un palíndromo, False en caso contrario. 
- Función para generar una lista de números primos hasta un límite: Recibe un límite como parámetro y devuelve una lista con los números primos encontrados.
- Función para calcular el mínimo común múltiplo (MCM) de dos números: Recibe dos números como parámetros y devuelve su MCM. 
- Función para calcular el salario de un empleado con horas extras: Recibe las horas trabajadas y la tarifa por hora como parámetros. Si las horas trabajadas superan las 40, calcula las horas extras con una tarifa del 150%. 

Ejercicios con listas y diccionarios: 
- Función para calcular la media de una lista de números: Recibe una lista de números como parámetro y devuelve su media.
- Función para contar la frecuencia de cada elemento en una lista: Recibe una lista como parámetro y devuelve un diccionario con la frecuencia de cada elemento.
- Función para encontrar la intersección de dos listas: Recibe dos listas como parámetros y devuelve una nueva lista con los elementos comunes.
- Función para ordenar una lista de números de forma ascendente: Recibe una lista como parámetro y devuelve la lista ordenada. 

Ejercicios con recursividad:
1. Función recursiva para calcular la secuencia de Fibonacci:
Recibe un número como parámetro y devuelve el término correspondiente de la secuencia de Fibonacci.
2. Función recursiva para buscar un elemento en una lista:
Recibe una lista y un elemento como parámetros y devuelve True si el elemento se encuentra en la lista, False en caso contrario. 

Consejos para resolver los ejercicios:

Comienza con la estructura básica:

Usa la palabra clave def para definir la función, seguida del nombre de la función y los parámetros entre paréntesis.

Define el cuerpo de la función:

Dentro de la función, escribe el código que realizará la tarea deseada.

Utiliza la palabra clave return para devolver el resultado:

Si la función debe devolver un valor, usa return seguido del valor a devolver.

Prueba la función con diferentes entradas:

Llama a la función con diferentes valores para asegurarte de que funciona correctamente.

Divide los problemas complejos en partes más pequeñas:

Si el problema es complejo, puedes dividirlo en varias funciones más pequeñas y luego combinarlas. 

Recuerda que la práctica es la clave para dominar la creación de funciones en Python. ¡Empieza con los ejercicios más sencillos y ve avanzando gradualmente! 

**Función para saludar**: Crea una función que reciba un nombre como parámetro y devuelva un saludo personalizado.

In [ ]:
def greeting(name:str) -> str:
    #name = str(input("Escriba el nombre"))
    saludo = f"Hola {name}"
    return saludo

In [6]:
print(greeting("Randolph"))

Hola Randolph


**Función para calcular el área de un círculo**: Recibe el radio como parámetro y devuelve el área.

In [5]:
def area(radio:float) -> float:
    #radio = float(input("Ingrese el radio"))
    area = radio**2
    return area

In [6]:
print(area(3))

9


**Función para verificar si un número es par**: Recibe un número como parámetro y devuelve True si es par, False en caso contrario.

In [7]:
def paridad(n):
    if n%2 == 0 or n==0:
        print('el numero es par')
    else:
        print('el numero es impar')

In [8]:
x = int(input("Ingrese su numero"))
paridad(x)

el numero es par


**Función para verificar si una cadena es un palíndromo**: Recibe una cadena como parámetro y devuelve True si es un palíndromo, False en caso contrario. 

In [9]:
def es_palindromo(cadena):
  """
  Verifica si una cadena es un palíndromo.

  Args:
    cadena: La cadena a verificar.

  Returns:
    True si la cadena es un palíndromo, False en caso contrario.
  """
  cadena = cadena.lower()
  cadena_invertida = cadena[::-1]
  return cadena == cadena_invertida

In [10]:
cadena1 = "madam"
cadena2 = "hello"

if es_palindromo(cadena1):
  print(f'"{cadena1}" es un palíndromo')
else:
  print(f'"{cadena1}" no es un palíndromo')

if es_palindromo(cadena2):
  print(f'"{cadena2}" es un palíndromo')
else:
  print(f'"{cadena2}" no es un palíndromo')

"madam" es un palíndromo
"hello" no es un palíndromo


**Función para generar una lista de números primos hasta un límite**: Recibe un límite como parámetro y devuelve una lista con los números primos encontrados.

In [1]:
def generar_primos(limite):
    """
    Genera una lista de números primos hasta un límite dado.

    Args:
        limite: El límite superior para buscar números primos.

    Returns:
        Una lista de números primos encontrados.
    """
    primos = []
    for num in range(2, limite + 1):
        es_primo = True
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                es_primo = False
                break
        if es_primo:
            primos.append(num)
    return primos

In [2]:
# Ejemplo de uso:
limite_maximo = 20
lista_primos = generar_primos(limite_maximo)
print(f"Números primos hasta {limite_maximo}: {lista_primos}")

Números primos hasta 20: [2, 3, 5, 7, 11, 13, 17, 19]


## Diseña un sistema de Autocompletado tipo Google Search
## 🧩 Problema
Implementa un sistema de autocompletado donde, dada una lista de frases populares y su frecuencia de búsqueda, puedas:

Registrar las frases ingresadas por el usuario.

Cuando el usuario ingresa un prefijo, el sistema debe sugerir las 3 frases más frecuentes que comiencen con ese prefijo.

Si hay empate en frecuencia, se ordenan alfabéticamente.

## 📥 Input
Lista de frases populares y su frecuencia:
Ejemplo:

python
Copiar
Editar

sentences = ["i love you", "island", "ironman", "i love leetcode"]

times = [5, 3, 2, 2]

El usuario ingresa letra por letra usando el método input(c):

Cada llamada con una letra actualiza las sugerencias.

Si se ingresa el carácter '#', la frase hasta ahora escrita se considera nueva entrada (se guarda o actualiza su frecuencia).

In [1]:
import heapq
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = {}
        self.counts = defaultdict(int)
        self.is_end = False

class AutocompleteSystem:
    def __init__(self, sentences, times):
        self.root = TrieNode()
        self.current_input = ""
        self.curr_node = self.root
        for sentence, time in zip(sentences, times):
            self._add(sentence, time)

    def _add(self, sentence, count):
        node = self.root
        for char in sentence:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
            node.counts[sentence] += count
        node.is_end = True

    def _lookup(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return {}
            node = node.children[char]
        return node.counts

    def input(self, c):
        if c == "#":
            self._add(self.current_input, 1)
            self.current_input = ""
            self.curr_node = self.root
            return []
        self.current_input += c
        counts = self._lookup(self.current_input)
        heap = [(-freq, sentence) for sentence, freq in counts.items()]
        heapq.heapify(heap)
        return [heapq.heappop(heap)[1] for _ in range(min(3, len(heap)))]

In [2]:
ac = AutocompleteSystem(
    ["i love you", "island", "ironman", "i love leetcode"],
    [5, 3, 2, 2]
)

print(ac.input('i'))  # ['i love you', 'i love leetcode', 'island']
#print(ac.input(' '))  # ['i love you', 'i love leetcode']
#print(ac.input('l'))  # ['i love you', 'i love leetcode']
#print(ac.input('o'))  # ['i love you']
#print(ac.input('#'))  # registra "i lo"


['i love you', 'island', 'i love leetcode']


**Función para encontrar el valor máximo de una lista**: Recibe una lista de números como parámetro y devuelve el valor máximo. 

In [1]:
def encontrar_maximo(lista):
  """
  Encuentra el valor máximo en una lista de números.

  Args:
    lista: Una lista de números.

  Returns:
    El valor máximo de la lista.
  """
  return max(lista)


In [2]:
# Ejemplo de uso
numeros = [1, 5, 2, 8, 3]
maximo = encontrar_maximo(numeros)
print(f"El valor máximo es: {maximo}")

El valor máximo es: 8


**Función para calcular el factorial de un número**: Recibe un número entero positivo y devuelve su factorial. 

In [1]:
def factorial(n):
    """
    Cacular el factorial de un numero 
    """
    result = 1
    for i in range(1,n+1):
        result *= i
    return result

In [2]:
x = int(input("Ingrese el numero"))
factorial(x)

720

## Ejercicios de datos simples

## Ejercicio 3
Escribir un programa que pregunte el nombre del usuario en la consola y después de que el usuario lo introduzca muestre por pantalla la cadena ¡Hola <nombre>!, donde <nombre> es el nombre que el usuario haya introducido.

In [1]:
name = str(input("Ingrese su nombre"))
print(f"Hola {name}")

Hola Alan


## Ejercicio 4

Escribir un programa que muestre por pantalla el resultado de la siguiente operación aritmética

<math xmlns="http://www.w3.org/1998/Math/MathML">
  <msup>
    <mrow data-mjx-texclass="INNER">
      <mo data-mjx-texclass="OPEN">(</mo>
      <mfrac>
        <mrow>
          <mn>3</mn>
          <mo>+</mo>
          <mn>2</mn>
        </mrow>
        <mrow>
          <mn>2</mn>
          <mo>&#x22C5;</mo>
          <mn>5</mn>
        </mrow>
      </mfrac>
      <mo data-mjx-texclass="CLOSE">)</mo>
    </mrow>
    <mn>2</mn>
  </msup>
</math>

In [2]:
print(((3+2)/(2*5))**2)

0.25


## Ejercicio 5
Escribir un programa que pregunte al usuario por el número de horas trabajadas y el coste por hora. Después debe mostrar por pantalla la paga que le corresponde.

In [3]:
hours = int(input("Cuantas horas trabajo?"))
cost = int(input("Precio de las"))
print(f"Lo correspondiente a pagar es la siguiente {hours*cost}")

Lo correspondiente a pagar es la siguiente 42000


## Ejercicio 6
Escribir un programa que lea un entero positivo <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>n</mi>
</math>, introducido por el usuario y después muestre en pantalla la suma de todos los enteros desde 1 hasta <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>n</mi>
</math>
. La suma de los <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>n</mi>
</math>
 primeros enteros positivos puede ser calculada de la siguiente forma:

 <math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mstyle displaystyle="false" scriptlevel="0">
    <mtext>suma</mtext>
  </mstyle>
  <mo>=</mo>
  <mfrac>
    <mrow>
      <mi>n</mi>
      <mo stretchy="false">(</mo>
      <mi>n</mi>
      <mo>+</mo>
      <mn>1</mn>
      <mo stretchy="false">)</mo>
    </mrow>
    <mn>2</mn>
  </mfrac>
</math>

In [6]:
x = int(input("Ingrese el numero entero"))
if x>=0:
    print((x*(x+1))/2)
else:
    print("No es un numero positivo")

No es un numero positivo


## Ejercicio 7
Escribir un programa que pida al usuario su peso (en kg) y estatura (en metros), calcule el índice de masa corporal y lo almacene en una variable, y muestre por pantalla la frase Tu índice de masa corporal es <imc> donde <imc> es el índice de masa corporal calculado redondeado con dos decimales.

In [9]:
weight = float(input("ingrese su peso en Kg"))
hight = float(input("ingrese su altura en M"))
imc = (weight/hight**2)
print(f"Su indice de masa corporal es de {imc}")

Su indice de masa corporal es de 24.845679012345677


## Ejercicio 8
Escribir un programa que pida al usuario dos números enteros y muestre por pantalla la n entre m da un cociente c y un resto r donde n y m son los números introducidos por el usuario, y c y r son el cociente y el resto de la división entera respectivamente.

In [1]:
n = float(input("ingrese el numero dividendo"))
m = float(input("ingrese el numero divisor"))
c = n/m
r = n%m
print(f"el numero {n} entre {m} da un cociente {c} y un resto {r} ")

el numero 4.0 entre 2.0 da un cociente 2.0 y un resto 0.0 


## Ejercicio 9
Escribir un programa que pregunte al usuario una cantidad a invertir, el interés anual y el número de años, y muestre por pantalla el capital obtenido en la inversión.

In [14]:
x = float(input("Cantidad a invertir"))
a = float(input("ingrese los años que desea invertir"))
interes = x*a*(1 - 0.2687)
print(f"Este es el resultado de la capital {interes}")

Este es el resultado de la capital 1462600.0000000002


## Ejercicio 10
Una juguetería tiene mucho éxito en dos de sus productos: payasos y muñecas. Suele hacer venta por correo y la empresa de logística les cobra por peso de cada paquete así que deben calcular el peso de los payasos y muñecas que saldrán en cada paquete a demanda. Cada payaso pesa 112 g y cada muñeca 75 g. Escribir un programa que lea el número de payasos y muñecas vendidos en el último pedido y calcule el peso total del paquete que será enviado.

In [3]:
peso_payaso = 112
peso_muñeca = 75
payasos = int(input("Introduce el número de payasos a enviar: "))
muñecas = int(input("Introduce el número de muñecas a enviar: "))
peso_total = peso_payaso * payasos + peso_muñeca * muñecas
print("El peso total del paquete es " + str(peso_total) + " g")

El peso total del paquete es 449 g


## Ejercicio 11
Imagina que acabas de abrir una nueva cuenta de ahorros que te ofrece el 4% de interés al año. Estos ahorros debido a intereses, que no se cobran hasta finales de año, se te añaden al balance final de tu cuenta de ahorros. Escribir un programa que comience leyendo la cantidad de dinero depositada en la cuenta de ahorros, introducida por el usuario. Después el programa debe calcular y mostrar por pantalla la cantidad de ahorros tras el primer, segundo y tercer años. Redondear cada cantidad a dos decimales.

En algunos casos, las cuentas de ahorros pueden calcular los intereses de forma diaria o mensual. En estos casos, la fórmula del interés simple se modifica para reflejar el período de tiempo más corto:

I = (P * r * t) / 365

Por ejemplo, si una cuenta de ahorros calcula los intereses de forma diaria y tiene una tasa de interés anual del 3%, los intereses generados al cabo de un día serán:

I = (1000 * 0.03 / 365)

I = $0.008

En este caso, los intereses generados al cabo de un año serán de $2.86.

In [ ]:
inversion = float(input("Introduce la inversión inicial: "))
interes = 0.04
balance1 = inversion * (1 + interes)
print("Balance tras el primer año:" + str(round(balance1, 2)))
balance2 = balance1 * (1 + interes)
print("Balance tras el segundo año:" + str(round(balance2, 2)))
balance3 = balance2 * (1 + interes)
print("Balance tras el tercer año:" + str(round(balance3, 2)))

## Ejercicio 12
Una panadería vende barras de pan a 3.49€ cada una. El pan que no es el día tiene un descuento del 60%. Escribir un programa que comience leyendo el número de barras vendidas que no son del día. Después el programa debe mostrar el precio habitual de una barra de pan, el descuento que se le hace por no ser fresca y el coste final total.

In [ ]:
barras = float(input("Ingrese el numero de barras"))
pan = 3.49
descuento = 0.6
coste = pan*barras*(1-descuento)
print(f"El precio de las barras es de {coste} €")
print("El descuento sobre una barra no fresca es " + str(descuento * 100) + "%")
print("El coste final a pagar es " + str(round(coste, 2)) + "€")

## Ejercicios de Cadenas

## Ejercicio 1
Escribir un programa que pregunte el nombre del usuario en la consola y un número entero e imprima por pantalla en líneas distintas el nombre del usuario tantas veces como el número introducido.

In [1]:
nombre = input("¿Cómo te llamas? ")
n = input("Introduce un número entero: ")
print((nombre + "\n") * int(n))

Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph



## Ejercicio 2
Escribir un programa que pregunte el nombre completo del usuario en la consola y después muestre por pantalla el nombre completo del usuario tres veces, una con todas las letras minúsculas, otra con todas las letras mayúsculas y otra solo con la primera letra del nombre y de los apellidos en mayúscula. El usuario puede introducir su nombre combinando mayúsculas y minúsculas como quiera.

In [6]:
complet_name = str(input("Introduzca su nombre completo"))
print(complet_name.upper() + '\n' + complet_name.lower() + '\n' + complet_name.title())

RANDOLPH RAFAEL PERALTA MONTERROZA
randolph rafael peralta monterroza
Randolph Rafael Peralta Monterroza


## Ejercicio 3
Escribir un programa que pregunte el nombre del usuario en la consola y después de que el usuario lo introduzca muestre por pantalla NOMBRE tiene n letras, donde NOMBRE es el nombre de usuario en mayúsculas y n es el número de letras que tienen el nombre.

In [1]:
N = str(input("Introduzca su nombre"))
n = len(N)
print(f"el nombre de {N.upper()} tiene {n} letras")

el nombre de RANDOLPH tiene 8 letras


## Ejercicio 4
Los teléfonos de una empresa tienen el siguiente formato prefijo-número-extension donde el prefijo es el código del país +34, y la extensión tiene dos dígitos (por ejemplo +34-913724710-56). Escribir un programa que pregunte por un número de teléfono con este formato y muestre por pantalla el número de teléfono sin el prefijo y la extensión.

In [4]:
tel = input("Introduce un número de teléfono con el formato +xx-xxxxxxxxx-xx: ")
print('El número de teléfono es ', tel[4:-3])

El número de teléfono es  913724710


## Ejercicio 5
Escribir un programa que pida al usuario que introduzca una frase en la consola y muestre por pantalla la frase invertida.

In [5]:
name = str(input("Ingrese su nombre"))
name_inverted = name[::-1]
print(name_inverted)

hplodnaR


## Ejercicio 6
Escribir un programa que pida al usuario que introduzca una frase en la consola y una vocal, y después muestre por pantalla la misma frase pero con la vocal introducida en mayúscula.

In [7]:
frase = str(input("Introduzca su frase"))
vocal = str(input("Introduzca una vocal"))
print(frase + " " + vocal.upper())

Fuerza A


## Ejercicio 7
Escribir un programa que pregunte el correo electrónico del usuario en la consola y muestre por pantalla otro correo electrónico con el mismo nombre (la parte delante de la arroba @) pero con dominio ceu.es.

In [2]:
email = input("Introduce tu correo electrónico: ")
print(email[:email.find('@')] + '@ceu.es')

asalbe@ceu.es


## Ejercicio 8
Escribir un programa que pregunte por consola el precio de un producto en euros con dos decimales y muestre por pantalla el número de euros y el número de céntimos del precio introducido.

In [14]:
precio = str(input("Introduce el precio del producto con dos decimales:  "))
print(f"Se tiene {precio[:precio.find(".")]} euros, donde  hay {precio[precio.find(".")+1:]} centimos")

Se tiene 123 euros, donde  hay 654 centimos


## Ejercicio 9
Escribir un programa que pregunte al usuario la fecha de su nacimiento en formato dd/mm/aaaa y muestra por pantalla, el día, el mes y el año. Adaptar el programa anterior para que también funcione cuando el día o el mes se introduzcan con un solo carácter.

In [14]:
fecha = str(input("ingrese su fecha de nacimiento"))
fecha = fecha.split("/")
dia, mes, año = fecha
print(f"Se tiene {dia} del mes {mes} del año {año}")

Se tiene 12 del mes 8 del año 2021


## Ejercicio 10
Escribir un programa que pregunte por consola por los productos de una cesta de la compra, separados por comas, y muestre por pantalla cada uno de los productos en una línea distinta.

In [20]:
productos = str(input("Introduzca los productos de su cesta de compras"))
productos = productos.split(",")
for producto in productos:
    print(producto)

manzana
 pera
 maracuya
 naranja


## Ejercicio 11
Escribir un programa que pregunte el nombre el un producto, su precio y un número de unidades y muestre por pantalla una cadena con el nombre del producto seguido de su precio unitario con 6 dígitos enteros y 2 decimales, el número de unidades con tres dígitos y el coste total con 8 dígitos enteros y 2 decimales.

In [ ]:
producto = str(input("Cual es su producto"))
precio = float(input("Cual es su precio"))
unidad = int(input("Cuantos desea"))
print('{producto}: {unidad:3d} unidad x {precio:09.2f}€ = {total:011.2f}€'.format(producto = producto, unidad = unidad, precio = precio, total = unidad * precio))

Manzana:   1 unidad x 000090.00€ = 00000090.00€


## Ejercicios de Condicionales

## Ejercicio 1
Escribir un programa que pregunte al usuario su edad y muestre por pantalla si es mayor de edad o no.

In [2]:
edad = int(input("ingrse su edad"))
if edad >= 18:
    print("Eres mayor de edad")
else:
    print("Eres menor de edad")

Eres mayor de edad


## Ejercicio 2
Escribir un programa que almacene la cadena de caracteres contraseña en una variable, pregunte al usuario por la contraseña e imprima por pantalla si la contraseña introducida por el usuario coincide con la guardada en la variable sin tener en cuenta mayúsculas y minúsculas.

In [3]:
key = "1234qwer"
password = str(input("introduzca la contraseña del usuario: "))
if password.lower() == key:
    print("Contraseña correcta")
else:
    print("Contraseña incorrecta")

Contraseña correcta


## Ejercicio 3
Escribir un programa que pida al usuario dos números y muestre por pantalla su división. Si el divisor es cero el programa debe mostrar un error.

In [7]:
dividendo = float(input("Ingrese su numero: "))
divisor = float(input("Ingrese el numero divisor: "))
if divisor != 0:
    print(f"La divison es {dividendo/divisor}")
else:
    print("El divisor es cero, ingrese nuevamnte la division")

La divison es 2.0


# Ejercicio 4
Escribir un programa que pida al usuario un número entero y muestre por pantalla si es par o impar.

In [16]:
numero = int(input("Ingrese su numero"))
if numero % 2 == 0:
    print(f"El numero {numero} par")
else:
    print(f"El numero {numero} impar")

El numero -9 impar


## Ejercicio 5
Para tributar un determinado impuesto se debe ser mayor de 16 años y tener unos ingresos iguales o superiores a 1000 € mensuales. Escribir un programa que pregunte al usuario su edad y sus ingresos mensuales y muestre por pantalla si el usuario tiene que tributar o no.

In [17]:
edad = int(input("ingrese su edad"))
ingresos = float(input("ingrese sus ingresos mensuales"))
if edad >= 16 and ingresos >= 1000:
    print("Debe tributar")
else:
    print("No debe tributar")

No debe tributar


## Ejercicio 6
Los alumnos de un curso se han dividido en dos grupos A y B de acuerdo al sexo y el nombre. El grupo A esta formado por las mujeres con un nombre anterior a la M y los hombres con un nombre posterior a la N y el grupo B por el resto. Escribir un programa que pregunte al usuario su nombre y sexo, y muestre por pantalla el grupo que le corresponde.

In [ ]:
name = input("¿Cómo te llamas? ")
gender = input("¿Cuál es tu sexo (M o H)? ")
if gender == "M":
    if name.lower() < "m":
        group = "A"
    else:
        group = "B"
else:
    if name.lower() > "n":
        group = "A"
    else:
        group = "B"
print("Tu grupo es " + group)

## Ejercicio 7
Los tramos impositivos para la declaración de la renta en un determinado país son los siguientes:

```
Renta	                Tipo impositivo
Menos de 10000€	             5%
Entre 10000€ y 20000€	    15%
Entre 20000€ y 35000€	    20%
Entre 35000€ y 60000€	    30%
Más de 60000€	            45%
```

Escribir un programa que pregunte al usuario su renta anual y muestre por pantalla el tipo impositivo que le corresponde.

In [ ]:
renta = float(input("ingrese su renta"))
if renta < 10000:
    tipo = 5
elif renta < 20000:
    tipo = 15
elif renta < 35000:
    tipo = 20
elif renta < 60000:
    tipo = 30
else:
    tipo = 45
# Mostrar por pantalla el producto de la renta por el tipo impositivo
print("Tienes que pagar ", renta * tipo / 100, "€", sep='')

14814.81


## Ejercicio 8
En una determinada empresa, sus empleados son evaluados al final de cada año. Los puntos que pueden obtener en la evaluación comienzan en 0.0 y pueden ir aumentando, traduciéndose en mejores beneficios. Los puntos que pueden conseguir los empleados pueden ser 0.0, 0.4, 0.6 o más, pero no valores intermedios entre las cifras mencionadas. A continuación se muestra una tabla con los niveles correspondientes a cada puntuación. La cantidad de dinero conseguida en cada nivel es de 2.400€ multiplicada por la puntuación del nivel.

```
Nivel	        Puntuación
Inaceptable	    0.0
Aceptable	    0.4
Meritorio	    0.6 o más
```

Escribir un programa que lea la puntuación del usuario e indique su nivel de rendimiento, así como la cantidad de dinero que recibirá el usuario.

In [5]:
bonificacion = 2400
inaceptable = 0
aceptable = 0.4
meritorio = 0.6
puntos = float(input("Introduce tu puntuación: "))
# Clasifiación por niveles de rendimiento
if puntos == inaceptable:
    nivel = "Inaceptable"
elif puntos == aceptable:
    nivel = "Aceptable"
elif puntos >= 0.6:
    nivel = "Meritorio"
else:
    nivel = ""
# Mostrar nivel de rendimiento
if nivel == "":
    print("Esta puntuación no es válida")
else:
    print("Tu nivel de rendimiento es %s" % nivel)
    print("Te corresponde cobrar %.2f€" % (puntos * bonificacion))

Tu nivel de rendimiento es Meritorio
Te corresponde cobrar 2400.00€


## Ejercicio 9
Escribir un programa para una empresa que tiene salas de juegos para todas las edades y quiere calcular de forma automática el precio que debe cobrar a sus clientes por entrar. El programa debe preguntar al usuario la edad del cliente y mostrar el precio de la entrada. Si el cliente es menor de 4 años puede entrar gratis, si tiene entre 4 y 18 años debe pagar 5€ y si es mayor de 18 años, 10€.

In [6]:
edad = int(input("Introduza su edad"))
if edad <= 0:
    print("No es una edad adecuada")
elif edad < 4:
    print("Entrada gratis")
elif edad < 18:
    print("Debe pagar 5€")
else:
    print("Debe pagar 10€")

Debe pagar 5€


## Ejercicio 10
La pizzería Bella Napoli ofrece pizzas vegetarianas y no vegetarianas a sus clientes. Los ingredientes para cada tipo de pizza aparecen a continuación.

- Ingredientes vegetarianos: Pimiento y tofu.
- Ingredientes no vegetarianos: Peperoni, Jamón y Salmón.

Escribir un programa que pregunte al usuario si quiere una pizza vegetariana o no, y en función de su respuesta le muestre un menú con los ingredientes disponibles para que elija. Solo se puede eligir un ingrediente además de la mozzarella y el tomate que están en todas la pizzas. Al final se debe mostrar por pantalla si la pizza elegida es vegetariana o no y todos los ingredientes que lleva.

In [12]:
# Presentación del menú con los tipos de pizza
print("Bienvenido a la pizzeria Bella Napoli.\nTipos de pizza\n\t1- Vegetariana\n\t2- No vegetariana\n")
pizza = input("Escoga su pizza:  1. vegetariana o 2. no vegetariana")
if pizza == "1":
    print("Escogio pizza vegetariana")
    ingrediente_v = input("Que ingrediente desea:  1. pimiento o 2. tofu.")
    if ingrediente_v == "1":
        print("Su pizza es vegetariana de mozarella, tomate y pimiento")
    else:
        print("Su pizza es vegetariana de mozarella, tomate y tofu")
else:
    print("Escogio pizza vegetariana")
    ingrediente = input("Que ingrediente desea:  1. peperoni, 2. jamón y 3. salmón.")
    if ingrediente == "1":
        print("Su pizza es vegetariana de mozarella, tomate y peperoni")
    elif ingrediente == "2":
        print("Su pizza es vegetariana de mozarella, tomate y jamón")
    elif ingrediente == "3":
        print("Su pizza es vegetariana de mozarella, tomate y salmón")



Bienvenido a la pizzeria Bella Napoli.
Tipos de pizza
	1- Vegetariana
	2- No vegetariana

Escogio pizza vegetariana
Su pizza es vegetariana de mozarella, tomate y jamón


## Ejercicios de Bucles

## Ejercicio 1
Escribir un programa que pida al usuario una palabra y la muestre por pantalla 10 veces.

In [17]:
name = str(input("ingrese su nombre: "))
for i in range(10):
    print(name)

Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph


## Ejercicio 2
Escribir un programa que pregunte al usuario su edad y muestre por pantalla todos los años que ha cumplido (desde 1 hasta su edad).

In [21]:
age = int(input("Cual es su edad"))
for i in range(age):
    print("Usted cumpio " + str(i+1) + " años")

Usted cumpio 1 años
Usted cumpio 2 años
Usted cumpio 3 años
Usted cumpio 4 años
Usted cumpio 5 años
Usted cumpio 6 años
Usted cumpio 7 años
Usted cumpio 8 años
Usted cumpio 9 años
Usted cumpio 10 años
Usted cumpio 11 años
Usted cumpio 12 años
Usted cumpio 13 años
Usted cumpio 14 años
Usted cumpio 15 años
Usted cumpio 16 años
Usted cumpio 17 años
Usted cumpio 18 años
Usted cumpio 19 años
Usted cumpio 20 años


## Ejercicio 3
Escribir un programa que pida al usuario un número entero positivo y muestre por pantalla todos los números impares desde 1 hasta ese número separados por comas.

In [3]:
n = int(input("Ingrese su numero entero positivo"))
if n <= 0:
    print("No es un entero positivo")
else:
    for i in range(1, n+1, 2):
        print(i, end = ", ")

1, 3, 5, 7, 

## Ejercicio 4
Escribir un programa que pida al usuario un número entero positivo y muestre por pantalla la cuenta atrás desde ese número hasta cero separados por comas.

In [1]:
n = int(input("Ingrese su numero entero positivo"))
if n <= 0:
    print("No es un entero positivo")
else:
    for i in range(n, 0, -1):
        print(i, end = ", ")

9, 8, 7, 6, 5, 4, 3, 2, 1, 

## Ejercicio 5
Escribir un programa que pregunte al usuario una cantidad a invertir, el interés anual y el número de años, y muestre por pantalla el capital obtenido en la inversión cada año que dura la inversión.

In [2]:
inversion = float(input("Ingrese su inversion: "))
interes = int(input("Ingrese su interes anual: "))
anios = int(input("Ingrese la cantidad de años: "))
for i in range(anios):
    inversion *= 1 + interes/100
    print("En " + str(i+1) + " Años obtendra en inversion " + str(inversion))

En 1 Años obtendra en inversion 1100.0
En 2 Años obtendra en inversion 1210.0
En 3 Años obtendra en inversion 1331.0
En 4 Años obtendra en inversion 1464.1000000000001


## Ejercicio 6
Escribir un programa que pida al usuario un número entero y muestre por pantalla un triángulo rectángulo como el de más abajo, de altura el número introducido.

```
*
**
***
****
*****
```

In [3]:
entero = int(input("Dame un entero positivo: "))
for i in range(entero):
    print(i*"*")


*
**
***
****
*****
******
*******


## Ejercicio 7
Escribir un programa que muestre por pantalla la tabla de multiplicar del 1 al 10.

In [8]:
for i in range(1, 10):
    for j in range(1, 10):
        print(i*j, end ="\t")
    print("")

1	2	3	4	5	6	7	8	9	
2	4	6	8	10	12	14	16	18	
3	6	9	12	15	18	21	24	27	
4	8	12	16	20	24	28	32	36	
5	10	15	20	25	30	35	40	45	
6	12	18	24	30	36	42	48	54	
7	14	21	28	35	42	49	56	63	
8	16	24	32	40	48	56	64	72	
9	18	27	36	45	54	63	72	81	


## Ejercicio 8
Escribir un programa que pida al usuario un número entero y muestre por pantalla un triángulo rectángulo como el de más abajo.

```
1
3 1
5 3 1
7 5 3 1
9 7 5 3 1
```

In [36]:
n = int(input("ingrese su numero entero"))
for i in range(1, n+1, 2):
    for j in range(i, 0, -2):
        print(j , end="\t")
    print(" ")

1	 
3	1	 
5	3	1	 
7	5	3	1	 
9	7	5	3	1	 


## Ejercicio 9
Escribir un programa que almacene la cadena de caracteres contraseña en una variable, pregunte al usuario por la contraseña hasta que introduzca la contraseña correcta.

In [45]:
key = "contraseña"
password =""
while password != key:
    password = str(input("Introduce la contraseña: "))
print("Contraseña correcta")

Contraseña correcta


## Ejercicio 10
Escribir un programa que pida al usuario un número entero y muestre por pantalla si es un número primo o no.

In [34]:
n = int(input("Introduce un número entero positivo mayor que 2: "))
for i in range(2, n):
    if n % i == 0:
        break
if (i + 1)  == n:
    print(str(n) + " es primo")
else: 
    print(str(n) + " no es primo")

9 no es primo


## Ejercicio 11
Escribir un programa que pida al usuario una palabra y luego muestre por pantalla una a una las letras de la palabra introducida empezando por la última.

In [9]:
words = str(input("Introducir una palabra: "))
print(f"Su palabra tiene {len(words)} letras")
for i in range(len(words)-1, -1, -1):
    print(words[i])

Su palabra tiene 7 letras
a
r
b
a
l
a
p


## Ejercicio 12
Escribir un programa en el que se pregunte al usuario por una frase y una letra, y muestre por pantalla el número de veces que aparece la letra en la frase.

In [15]:
frase = str(input("ingrese su frase: "))
letra = str(input("ingrese su letra: "))
numero = 0
for i in range(0, len(frase)):
    if letra == frase[i]:
        numero += 1
print(f"la letra {letra} aparece {numero} veces en la {frase}")

la letra a aparece 3 veces en la palabra


## Ejercicio 13
Escribir un programa que muestre el eco de todo lo que el usuario introduzca hasta que el usuario escriba “salir” que terminará.

In [ ]:
salida = "Salir"
eco = ""
while eco != salida:
    eco = str(input("Introduce la frase: "))
    print(eco)
print("Finalizar")

Hola
tumae
JAJAJA
Salir
Finalizar


## Ejercicios de Listas y Tuplas

## Ejercicio 1
Escribir un programa que almacene las asignaturas de un curso (por ejemplo Matemáticas, Física, Química, Historia y Lengua) en una lista y la muestre por pantalla.

In [4]:
lista = []
asignatura = ""
while asignatura == "" :
    asignatura = str(input("Ingrese su asignatura "))
lista.append(asignatura)
lista

['Matemáticas, Física, Química, Historia y Lengua']

## Ejercicio 2
Escribir un programa que almacene las asignaturas de un curso (por ejemplo Matemáticas, Física, Química, Historia y Lengua) en una lista y la muestre por pantalla el mensaje: Yo estudio "asignatura", donde "asignatura" es cada una de las asignaturas de la lista.

In [5]:
asignaturas = ["Matemáticas", "Física", "Química", "Historia", "Lengua"]
for asignatura in asignaturas:
    print(f"Yo estudio {asignatura}, donde {asignatura} es cada una de las asignaturas de la lista.")

Yo estudio Matemáticas, donde Matemáticas es cada una de las asignaturas de la lista.
Yo estudio Física, donde Física es cada una de las asignaturas de la lista.
Yo estudio Química, donde Química es cada una de las asignaturas de la lista.
Yo estudio Historia, donde Historia es cada una de las asignaturas de la lista.
Yo estudio Lengua, donde Lengua es cada una de las asignaturas de la lista.


## Ejercicio 3
Escribir un programa que almacene las asignaturas de un curso (por ejemplo Matemáticas, Física, Química, Historia y Lengua) en una lista, pregunte al usuario la nota que ha sacado en cada asignatura, y después las muestre por pantalla con el mensaje. En "asignatura" has sacado "nota" donde "asignatura" es cada una des las asignaturas de la lista y "nota" cada una de las correspondientes notas introducidas por el usuario.

In [ ]:
asignaturas = ["Matemáticas", "Física", "Química", "Historia", "Lengua"]
nota = []
for asignatura in asignaturas:
    nota = int(input(f"Que saco en la asignatura {asignatura}"))
    print(f"En {asignatura} has sacado {nota}")

En Matemáticas has sacado 5
En Física has sacado 4
En Química has sacado 5
En Historia has sacado 4
En Lengua has sacado 5


## Ejercicio 4
Escribir un programa que pregunte al usuario los números ganadores de la lotería primitiva, los almacene en una lista y los muestre por pantalla ordenados de menor a mayor.

In [1]:
awarded = []
for i in range(6):
    awarded.append(int(input("Ingrese los numero de loteria ganadores: ")))
awarded.sort()
print(f"Los numero ganadores son: {awarded}")

Los numero ganadores son: [123, 321, 345, 456, 3456, 7765]


## Ejercicio 5
Escribir un programa que almacene en una lista los números del 1 al 10 y los muestre por pantalla en orden inverso separados por comas.

In [4]:
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in range(1, 11):
    print(numbers[-i], end=", ")

10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 

## Ejercicio 6
Escribir un programa que almacene las asignaturas de un curso (por ejemplo Matemáticas, Física, Química, Historia y Lengua) en una lista, pregunte al usuario la nota que ha sacado en cada asignatura y elimine de la lista las asignaturas aprobadas. Al final el programa debe mostrar por pantalla las asignaturas que el usuario tiene que repetir.

In [14]:
asignaturas = ["Matemáticas", "Física", "Química", "Historia", "Lengua"]

In [15]:

pasadas = []
for asignatura in asignaturas:
    nota = int(input(f"Asigne la nota de la asignatura {asignatura}: "))
    if nota >= 3:
        pasadas.append(asignatura)
        print(f"En {asignatura} tienes esta nota: {nota} y fue aprobada")
    else:
        print(f"Debes repetir la asignatura {asignatura}")
for asignatura in pasadas:
    asignaturas.remove(asignatura)

En Matemáticas tienes esta nota: 5 y fue aprobada
En Física tienes esta nota: 4 y fue aprobada
En Química tienes esta nota: 3 y fue aprobada
Debes repetir la asignatura Historia
Debes repetir la asignatura Lengua


## Ejercicio 7
Escribir un programa que almacene el abecedario en una lista, elimine de la lista las letras que ocupen posiciones múltiplos de 3, y muestre por pantalla la lista resultante.

In [8]:
abcdario = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
for i in abcdario:
    if abcdario.index(i) % 3 == 0:
        print(f"la letra {i} se elimino, porque tiene indice {abcdario.index(i)}")
abcdario.remove(i)

la letra a se elimino, porque tiene indice 0
la letra d se elimino, porque tiene indice 3
la letra g se elimino, porque tiene indice 6
la letra j se elimino, porque tiene indice 9


## Ejercicio 8
Escribir un programa que pida al usuario una palabra y muestre por pantalla si es un palíndromo.

In [15]:
palabra = str(input("Ingrese su palabra: "))
arbalap = palabra[::-1]
if palabra == arbalap:
    print(f"La palabra {palabra} es un palindromo")
else:
    print(f"La palabra {palabra} no es un palindromo")

La palabra arañara es un palindromo


## Ejercicio 9
Escribir un programa que pida al usuario una palabra y muestre por pantalla el número de veces que contiene cada vocal.

In [ ]:
palabra = str(input("Ingrese su palabra: "))
letra = str(input("Ingrese la letra: "))
conteo = 0
for i in palabra:
    if letra == i:
        conteo += 1
print(f"La letra {letra} aparece {conteo} veces, en la palabra {palabra}")

La letra a aparece 2 veces, en la palabra amar


## Ejercicio 10
Escribir un programa que almacene en una lista los siguientes precios, 50, 75, 46, 22, 80, 65, 8, y muestre por pantalla el menor y el mayor de los precios.

In [2]:
precios = [50, 75, 46, 22, 80, 65, 8]
precios.sort()
print(f"El menor precio es {precios[0]} y el mayor precio es {precios[-1]}")

El menor precio es 8 y el mayor precio es 80


In [3]:
prices = [50, 75, 46, 22, 80, 65, 8]
min = max = prices[0]
for price in prices:
    if price < min:
        min = price
    elif price > max:
        max = price 
print("El mínimo es " + str(min)) 
print("El máximo es " + str(max))

El mínimo es 8
El máximo es 80


## Ejercicio 11
Escribir un programa que almacene los vectores (1,2,3) y (-1,0,2) en dos listas y muestre por pantalla su producto escalar.

In [18]:
u1 = (1,2,3)
u2 = (-1,0,2)
v1 = list(u1)
v2 = list(u2)
k = 0
for i in v1:
    for j in v2:
        if v1.index(i) == v2.index(j):
            k += i*j
print(k)

5


## Ejercicio 12
Escribir un programa que almacene las matrices

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>A</mi>
  <mo>=</mo>
  <mrow data-mjx-texclass="INNER">
    <mo data-mjx-texclass="OPEN">(</mo>
    <mtable columnalign="right right right" columnspacing="1em" rowspacing="4pt">
      <mtr>
        <mtd>
          <mn>1</mn>
        </mtd>
        <mtd>
          <mn>2</mn>
        </mtd>
        <mtd>
          <mn>3</mn>
        </mtd>
      </mtr>
      <mtr>
        <mtd>
          <mn>4</mn>
        </mtd>
        <mtd>
          <mn>5</mn>
        </mtd>
        <mtd>
          <mn>6</mn>
        </mtd>
      </mtr>
    </mtable>
    <mo data-mjx-texclass="CLOSE">)</mo>
  </mrow>
  <mstyle scriptlevel="0">
    <mspace width="1em"></mspace>
  </mstyle>
  <mi>y</mi>
  <mstyle scriptlevel="0">
    <mspace width="1em"></mspace>
  </mstyle>
  <mi>B</mi>
  <mo>=</mo>
  <mrow data-mjx-texclass="INNER">
    <mo data-mjx-texclass="OPEN">(</mo>
    <mtable columnalign="right right" columnspacing="1em" rowspacing="4pt">
      <mtr>
        <mtd>
          <mo>&#x2212;</mo>
          <mn>1</mn>
        </mtd>
        <mtd>
          <mn>0</mn>
        </mtd>
      </mtr>
      <mtr>
        <mtd>
          <mn>0</mn>
        </mtd>
        <mtd>
          <mn>1</mn>
        </mtd>
      </mtr>
      <mtr>
        <mtd>
          <mn>1</mn>
        </mtd>
        <mtd>
          <mn>1</mn>
        </mtd>
      </mtr>
    </mtable>
    <mo data-mjx-texclass="CLOSE">)</mo>
  </mrow>
</math>
 
en una lista y muestre por pantalla su producto.
Nota: Para representar matrices mediante listas usar listas anidadas, representando cada vector fila en una lista.

In [ ]:
a = ((1,2,3),(4,5,6))
b = ((-1,0), (0,1), (1,1))
result = [[0,0], [0,0]]

for i in range(len(a)):
    for j in range(len(b[0])):
        for k in range(len(b)):
            result[i][j] += (a[i][k]*b[k][j])

for i in range(len(result)):
    result[i] = tuple(result[i])
result = tuple(result)
for i in range(len(result)):
    print(result[i])

(2, 5)
(2, 11)


## Ejercicio 13
Escribir un programa que pregunte por una muestra de números, separados por comas, los guarde en una lista y muestre por pantalla su media y desviación típica.

In [8]:
lista = input("Inrese su lista de numeros por comas: ")
lista = lista.split(",")
n = len(lista)
for p in range(n):
    lista[p] = int(lista[p])
j = 0
m = 0
for i in lista:
    j += i
u = j/len(lista)

for k in lista:
    m += (k-u)**2
d = m/len(lista)

print(f"Su media es de {u} y su desviacion tipica es {d}")

Su media es de 5.0 y su desviacion tipica es 6.666666666666667


## Ejercicios de diccionarios

## Ejercicio 1
Escribir un programa que guarde en una variable el diccionario {'Euro':'€', 'Dollar':'$', 'Yen':'¥'}, pregunte al usuario por una divisa y muestre su símbolo o un mensaje de aviso si la divisa no está en el diccionario.

In [1]:
monedas = {'Euro':'€', 'Dollar':'$', 'Yen':'¥'}
moneda = input("Introduce una divisa: ")
print(monedas.get(moneda.title(), "La divisa no está."))

¥


## Ejercicio 2
Escribir un programa que pregunte al usuario su nombre, edad, dirección y teléfono y lo guarde en un diccionario. Después debe mostrar por pantalla el mensaje <nombre> tiene <edad> años, vive en <dirección> y su número de teléfono es <teléfono>.

In [2]:
nombre = str(input("Cual es tu nombre?"))
edad = int(input("Cual es tu edad?"))
direccion = str(input("Cual es tu direccion?"))
telefono = int(input("Cual es tu numero de telefono o celular"))
persona = {'nombre': nombre, 'edad': edad, 'direccion': direccion, 'telefono':telefono}
print(f"Te llamas {persona['nombre']}, tu edad es de {persona['edad']} vives en esta direccion {persona['direccion']} y tu telefono es {persona['telefono']}")

Te llamas Randolph, tu edad es de 29 vives en esta direccion Call 32C #17-75 y tu telefono es 3013587938


## Ejercicio 3
Escribir un programa que guarde en un diccionario los precios de las frutas de la tabla, pregunte al usuario por una fruta, un número de kilos y muestre por pantalla el precio de ese número de kilos de fruta. Si la fruta no está en el diccionario debe mostrar un mensaje informando de ello.

```
Fruta	Precio
Plátano	 1.35
Manzana	 0.80
Pera	 0.85
Naranja	 0.70
```

In [3]:
fruta = str(input("Ingrese la fruta"))
cantidad = int(input("Cuantos kilos desea llevar"))

frutas = {'Platano':1.35, 'Manzana':0.80, 'Pera':0.85, 'Naranja':0.70}
if fruta in frutas:
    precio = frutas[fruta]
    total = cantidad * precio
    print(f"El precio de {cantidad} kilos de {fruta} es: ${total:.2f}")
else:
    print("la fruta no esta en la lista")

El precio de 2 kilos de Manzana es: $1.60


## Ejercicio 4
Escribir un programa que pregunte una fecha en formato dd/mm/aaaa y muestre por pantalla la misma fecha en formato dd de <mes> de aaaa donde <mes> es el nombre del mes.

In [7]:
meses = {
    '01': 'enero',
    '02': 'febrero',
    '03': 'marzo',
    '04': 'abril',
    '05': 'mayo',
    '06': 'junio',
    '07': 'julio',
    '08': 'agosto',
    '09': 'septiembre',
    '10': 'octubre',
    '11': 'noviembre',
    '12': 'diciembre'
}

fecha = input("Ingrese la fecha en formato dd/mm/aaaa: ")
partes = fecha.split('/')

if len(partes) == 3 and partes[1] in meses:
    dia, mes, año = partes
    print(f"{dia} de {meses[mes]} de {año}")
else:
    print("Formato de fecha inválido o mes no reconocido.")

01 de enero de 2001


## Ejercicio 5
Escribir un programa que almacene el diccionario con los créditos de las asignaturas de un curso {'Matemáticas': 6, 'Física': 4, 'Química': 5} y después muestre por pantalla los créditos de cada asignatura en el formato <asignatura> tiene <créditos> créditos, donde <asignatura> es cada una de las asignaturas del curso, y <créditos> son sus créditos. Al final debe mostrar también el número total de créditos del curso.

In [5]:
curso = {'Matemáticas': 6, 'Física': 4, 'Química': 5}
total_creditos = 0
for asignatura, creditos in curso.items():
    print(asignatura, 'tiene', creditos, 'créditos')
    total_creditos += creditos
print('Número total de créditos del curso: ', total_creditos)

Matemáticas tiene 6 créditos
Física tiene 4 créditos
Química tiene 5 créditos
Número total de créditos del curso:  15


## Ejercicio 6
Escribir un programa que cree un diccionario vacío y lo vaya llenado con información sobre una persona (por ejemplo nombre, edad, sexo, teléfono, correo electrónico, etc.) que se le pida al usuario. Cada vez que se añada un nuevo dato debe imprimirse el contenido del diccionario.

In [ ]:
persona= {}
continuar = True
while continuar:
    clave = input("Ingrese el dato que desea guardar")
    valor = input(clave + ': ')
    persona[clave] = valor
    print(persona)
    conitnuar = input("Deseas continuar Si / No?")
    if continuar == 'Si':
        continue
    else:
        break

{'Nombre': 'Randolph'}


## Ejercicio 7
Escribir un programa que cree un diccionario simulando una cesta de la compra. El programa debe preguntar el artículo y su precio y añadir el par al diccionario, hasta que el usuario decida terminar. Después se debe mostrar por pantalla la lista de la compra y el coste total, con el siguiente formato

```
Lista de la compra	

Artículo 1	Precio
Artículo 2	Precio
Artículo 3	Precio
…	…  ... ... ...
Total	    Coste
```

In [1]:
cesta = {}
continuar = True
while continuar:
    item = input('Introduce un artículo: ')
    precio = float(input('Introduce el precio de ' + item + ': '))
    cesta[item] = precio
    continuar = input('¿Quieres añadir artículos a la lista (Si/No)? ') == "Si"
coste = 0
print('Lista de la compra')
for item, precio in cesta.items():
    print(item, '\t', precio)
    coste += precio
print('Coste total: ', coste)

Lista de la compra
Mesa 	 100.0
Silla 	 200.0
Coste total:  300.0


In [1]:
cesta = {}

bandera = "s"
total = 0
while bandera == "s":
    articulo = str(input("Nombre del articulo: \n"))
    precio = float(input("Precio del articulo: $"))

    cesta[articulo] = precio

    bandera = str(input("Quiere agregar este articulo? prsiona (s) de lo contrario (n)"))

for i in cesta:
    print(i + "-------------" + str(cesta[i]))

Plato-------------500.0


## Ejercicio 8
Escribir un programa que cree un diccionario de traducción español-inglés. El usuario introducirá las palabras en español e inglés separadas por dos puntos, y cada par palabra:traducción separados por comas. El programa debe crear un diccionario con las palabras y sus traducciones. Después pedirá una frase en español y utilizará el diccionario para traducirla palabra a palabra. Si una palabra no está en el diccionario debe dejarla sin traducir.

In [1]:
diccionario = {}
palabras = input("Introduce la lista de palabras y traducciones en formato palabra:traducción separadas por comas: ")
for i in palabras.split(','):
    clave, valor = i.split(':')
    diccionario[clave] = valor
frase = input('Introduce una frase en español: ')
for i in frase.split():
    if i in diccionario:
        print(diccionario[i], end=' ')
    else:
        print(i, end=' ')

 Red 

## Ejercicio 9
Escribir un programa que gestione las facturas pendientes de cobro de una empresa. Las facturas se almacenarán en un diccionario donde la clave de cada factura será el número de factura y el valor el coste de la factura. El programa debe preguntar al usuario si quiere añadir una nueva factura, pagar una existente o terminar. Si desea añadir una nueva factura se preguntará por el número de factura y su coste y se añadirá al diccionario. Si se desea pagar una factura se preguntará por el número de factura y se eliminará del diccionario. Después de cada operación el programa debe mostrar por pantalla la cantidad cobrada hasta el momento y la cantidad pendiente de cobro.

In [2]:
facturas = {}
bandera = "s"
pendiente = 0
cobrado = 0

while bandera == "s":
    op = input("MENU\nQue accion deseas realizar:\n1-Agregar factura\n2-Pagar factura\n3.-SALIR")

    if op == "1":
        numf = int(input("AGREGAR FACTURA\nNumero de factura:\n"))
        valor = float(input("Importe de factura:\n"))
        facturas[numf] = valor
        pendiente += valor
        print("Pendiente: $" + str(pendiente))
        print("Cobrado el dia de hoy: $" + str(cobrado))

    elif op == "2":
        numf = int(input("PAGAR FACTURA\nEscribe el numero de factura:\n"))
        if numf in facturas:
            pendiente -= facturas[numf]
            cobrado += facturas[numf]
        else:
            print("ERROR: No se encontro la factura")
            
        print("Pendiente: $" + str(pendiente))
        print("Cobrado el dia de hoy: $" + str(cobrado))

    elif op == "3":
        print("Pendiente: $" + str(pendiente))
        print("Cobrado el dia de hoy: $" + str(cobrado) + "\n\n")
        bandera = "n"

Pendiente: $5432.0
Cobrado el dia de hoy: $0
Pendiente: $5432.0
Cobrado el dia de hoy: $0




## Ejercicio 10
Escribir un programa que permita gestionar la base de datos de clientes de una empresa. Los clientes se guardarán en un diccionario en el que la clave de cada cliente será su NIF, y el valor será otro diccionario con los datos del cliente (nombre, dirección, teléfono, correo, preferente), donde preferente tendrá el valor True si se trata de un cliente preferente. El programa debe preguntar al usuario por una opción del siguiente menú: (1) Añadir cliente, (2) Eliminar cliente, (3) Mostrar cliente, (4) Listar todos los clientes, (5) Listar clientes preferentes, (6) Terminar. En función de la opción elegida el programa tendrá que hacer lo siguiente:

1. Preguntar los datos del cliente, crear un diccionario con los datos y añadirlo a la base de datos.
2. Preguntar por el NIF del cliente y eliminar sus datos de la base de datos.
3. Preguntar por el NIF del cliente y mostrar sus datos.
4. Mostrar lista de todos los clientes de la base datos con su NIF y nombre.
5. Mostrar la lista de clientes preferentes de la base de datos con su NIF y nombre.
6. Terminar el programa.

In [ ]:
clientes = {}
bandera = "s"

while bandera == "s":
    op = input("MENU\nQue opcion deseas realizar\n(1)Agregar cliente\n(2)Eliminar cliente\n(3)Muestre cliente\n(4)Listar todos los clientes\n(5)Listar clientes referentes\n(6)Salir")

    if op == "1":
        print("opcion 1")
        nif = input("NIF: ")
        nombre = input("Nombre")
        direccion = input("Direccion")
        telefono = input("Telefono")
        correo = input("Correo")
        preferente = input("Es usuari prefente?(true/false): ")
        preferente = preferente.lower()
        cliente = {"nombre": nombre, "direccion": correo, "telefono": telefono, "correo": correo, "preferente": preferente}
        cliente[nif] = cliente
    
    elif op == "2":
        print("opcion 2")
        nif = input("\nELIMINAR USUARIO\nEscribir el NIF del usuario:\n")
        if nif in clientes:
            for clave, valor in clientes[nif].items():
                print(clave + " : " + clave)
                del(clientes[nif])
        else:
            print("Este usuario no existe\n")
        print("USUARIO ELIMINADO\n")
        
    elif op == "3":
        print("opcion 3")
        nif = input("\nBUSCAR CLIENTE\nEscribe el NIF del usuario:\n")
        if nif in clientes:
            for clave, valor in clientes[nif].items():
                print(clave + " : " + valor)
        else:
            print("Este usuario no existe\n")
        print("\n")
    
    elif op == "4":
        for nif in clientes:
                print("-----------------")
                for clave, valor in clientes[nif].items():
                    print(clave + " : " + valor)


    elif op == "5":
        print("opcion 5")
        for j in clientes:
            for k in clientes[j].items():
                if "true" in k:
                    for y in clientes[j].items():
                        if "nombre" in y:
                            clave = y[0].title()
                            valor = y[1].title()
                            print("USUARIO VIP: " + j + clave + ":" + valor + "\n")
        
    
    elif op == "6":
        bandera = "n"


opcion 1
opcion 2
Este usuario no existe

USUARIO ELIMINADO



In [ ]:
clientes = {}
opcion = ''
while opcion != '6':
    if opcion == '1':
        nif = input('Introduce NIF del cliente: ')
        nombre = input('Introduce el nombre del cliente: ')
        direccion = input('Introduce la dirección del cliente: ')
        telefono = input('Introduce el teléfono del cliente: ')
        email = input('Introduce el correo electrónico del cliente: ')
        vip = input('¿Es un cliente preferente (S/N)? ')
        cliente = {'nombre':nombre, 'dirección':direccion, 'teléfono':telefono, 'email':email, 'preferente':vip=='S'}
        clientes[nif] = cliente
    if opcion == '2':
        nif = input('Introduce NIF del cliente: ')
        if nif in clientes:
            del clientes[nif]
        else:
            print('No existe el cliente con el nif', nif)
    if opcion == '3':
        nif = input('Introduce NIF del cliente: ')
        if nif in clientes:
            print('NIF:', nif)
            for clave, valor in clientes[nif].items():
                print(clave.title() + ':', valor)
        else:
            print('No existe el cliente con el nif', nif)
    if opcion == '4':
        print('Lista de clientes')
        for clave, valor in clientes.items():
            print(clave, valor['nombre'])
    if opcion == '5':
        print('Lista de clientes preferentes')
        for clave, valor in clientes.items():
            if valor['preferente']:
                print(clave, valor['nombre'])
    opcion = input('Menú de opciones\n(1) Añadir cliente\n(2) Eliminar cliente\n(3) Mostrar cliente\n(4) Listar clientes\n(5) Listar clientes preferentes\n(6) Terminar\nElige una opción:')


## Ejercicio 11
El directorio de los clientes de una empresa está organizado en una cadena de texto como la de más abajo, donde cada línea contiene la información del nombre, email, teléfono, nif, y el descuento que se le aplica. Las líneas se separan con el carácter de cambio de línea \n y la primera línea contiene los nombres de los campos con la información contenida en el directorio.

"nif;nombre;email;teléfono;descuento\n01234567L;Luis González;luisgonzalez@mail.com;656343576;12.5\n71476342J;Macarena Ramírez;macarena@mail.com;692839321;8\n63823376M;Juan José Martínez;juanjo@mail.com;664888233;5.2\n98376547F;Carmen Sánchez;carmen@mail.com;667677855;15.7"

Escribir un programa que genere un diccionario con la información del directorio, donde cada elemento corresponda a un cliente y tenga por clave su nif y por valor otro diccionario con el resto de la información del cliente. Los diccionarios con la información de cada cliente tendrán como claves los nombres de los campos y como valores la información de cada cliente correspondientes a los campos. Es decir, un diccionario como el siguiente

{'01234567L': {'nombre': 'Luis González', 'email': 'luisgonzalez@mail.com', 'teléfono': '656343576', 'descuento': 12.5}, '71476342J': {'nombre': 'Macarena Ramírez', 'email': 'macarena@mail.com', 'teléfono': '692839321', 'descuento': 8.0}, '63823376M': {'nombre': 'Juan José Martínez', 'email': 'juanjo@mail.com', 'teléfono': '664888233', 'descuento': 5.2}, '98376547F': {'nombre': 'Carmen Sánchez', 'email': 'carmen@mail.com', 'teléfono': '667677855', 'descuento': 15.7}}

In [2]:
directorioCad = "nif;nombre;email;teléfono;descuento\n01234567L;Luis González;luisgonzalez@mail.com;656343576;12.5\n71476342J;Macarena Ramírez;macarena@mail.com;692839321;8\n63823376M;Juan José Martínez;juanjo@mail.com;664888233;5.2\n98376547F;Carmen Sánchez;carmen@mail.com;667677855;15.7"
DirectorioD = {}

lista_clientes = directorioCad.split("\n")
lista_campos = lista_clientes[0].split(";")

for i in lista_clientes[1:]:
    diccionario_cliente = {}
    lista_cliente = i.split(";")
    for j in range(len(lista_cliente)):
        if j == 0:
            nif = lista_cliente[j]
    else:
        diccionario_cliente[lista_campos[j]] = lista_cliente[j]
    DirectorioD[nif] = diccionario_cliente

print(DirectorioD)    

{'01234567L': {'descuento': '12.5'}, '71476342J': {'descuento': '8'}, '63823376M': {'descuento': '5.2'}, '98376547F': {'descuento': '15.7'}}


## -- Ejercicios de diccionarios
## Ejercicio 1 Permalink
Escribe un programa python que pida un número por teclado y que cree un diccionario cuyas claves sean desde el número 1 hasta el número indicado, y los valores sean los cuadrados de las claves.

In [4]:
numero = int(input("Escriba un numero"))
diccionario ={}

for i in range(1, numero+1):
    diccionario[i] = i*i

print(diccionario)

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36}


## Ejercicio 2 Permalink
Escribe un programa que lea una cadena y devuelva un diccionario con la cantidad de apariciones de cada carácter en la cadena.

In [9]:
texto = input("introduce el texto: ")
diccionario = {}

for elemento in texto:
    if elemento in diccionario:
        diccionario[elemento] += 1
    else:
        diccionario[elemento] = 1

print(f"La palabra que ingresaste fue {texto} y este es tu diccionario {diccionario}")

La palabra que ingresaste fue pitagoras y este es tu diccionario {'p': 1, 'i': 1, 't': 1, 'a': 2, 'g': 1, 'o': 1, 'r': 1, 's': 1}


## Ejercicio 3 Permalink
Vamos a crear un programa en python donde vamos a declarar un diccionario para guardar los precios de las distintas frutas. El programa pedirá el nombre de la fruta y la cantidad que se ha vendido y nos mostrará el precio final de la fruta a partir de los datos guardados en el diccionario. Si la fruta no existe nos dará un error. Tras cada consulta el programa nos preguntará si queremos hacer otra consulta.

In [10]:
diccionario = {"manzana": 500, "pera": 200, "uva": 300}
bandera = "s"
costo = 0
while bandera == "s":
   fruta = input("Que fruta desea comprar: ")
   cantidad = int(input("Que cantidad desea comprar: "))
   if fruta in diccionario:
      costo += diccionario[fruta]*cantidad
   else:
      print("la fruta no esta en el diccionario")
   

   valor = input("Deseas continuar para hacer otra consulta: s/n")
   bandera = valor
   print(f"Ha comprado {cantidad} {fruta} su costo es de {costo}")
print(f"El total de su compra es {costo}")

Ha comprado 1 manzana su costo es de 500
Ha comprado 4 pera su costo es de 1300
Ha comprado 6 uva su costo es de 3100
El total de su compra es 3100


## Ejercicio 4 Permalink
Codifica un programa en python que nos permita guardar los nombres de los alumnos de una clase y las notas que han obtenido. Cada alumno puede tener distinta cantidad de notas. Guarda la información en un diccionario cuya claves serán los nombres de los alumnos y los valores serán listas con las notas de cada alumno.

El programa pedirá el número de alumnos que vamos a introducir, pedirá su nombre e irá pidiendo sus notas hasta que introduzcamos un número negativo. Al final el programa nos mostrará la lista de alumnos y la nota media obtenida por cada uno de ellos. Nota: si se introduce el nombre de un alumno que ya existe el programa nos dará un error.

In [ ]:
alumnos = {}

try:
    num_alumnos = int(input("¿Cuántos alumnos vas a introducir? "))
except ValueError:
    print("Por favor, introduce un número válido.")

for i in range(n):
        nombre = input(f"\nIntroduce el nombre del alumno #{i + 1}: ").strip()
        if nombre in alumnos:
            print("❌ Error: El alumno ya existe. No se puede introducir dos veces.")
            continue

        notas = []
        while True:
            try:
                nota = float(input(f"Introduce una nota para {nombre} (número negativo para terminar): "))
                if nota < 0:
                    break
                notas.append(nota)
            except ValueError:
                print("⚠️ Por favor, introduce una nota válida.")

        alumnos[nombre] = notas
        
alumnos

In [ ]:
nombre = input("Nombre del alumno: ")
nota = int(input("Mencione su nota: "))

diccionario = {}

if nombre in diccionario:
    diccionario[nombre].append(nota)
else:
    diccionario[nombre] = [nota]

print(diccionario)

{'Jose': [1, 2]}


In [ ]:
def main():
    alumnos = {}
    try:
        num_alumnos = int(input("¿Cuántos alumnos vas a introducir? "))
    except ValueError:
        print("Por favor, introduce un número válido.")
        return

    for i in range(num_alumnos):
        nombre = input(f"\nIntroduce el nombre del alumno #{i + 1}: ").strip()
        if nombre in alumnos:
            print("❌ Error: El alumno ya existe. No se puede introducir dos veces.")
            continue

        notas = []
        while True:
            try:
                nota = float(input(f"Introduce una nota para {nombre} (número negativo para terminar): "))
                if nota < 0:
                    break
                notas.append(nota)
            except ValueError:
                print("⚠️ Por favor, introduce una nota válida.")

        alumnos[nombre] = notas

    print("\n📊 Resultados:")
    for nombre, notas in alumnos.items():
        if notas:
            media = sum(notas) / len(notas)
            print(f"{nombre}: Media = {media:.2f}")
        else:
            print(f"{nombre}: No se introdujeron notas.")

if __name__ == "__main__":
    main()

## Ejercicio 5 Permalink
Escribir un programa que implemente una agenda. En la agenda se podrán guardar nombres y números de teléfono. El programa nos dará el siguiente menú:

- Añadir/modificar: Nos pide un nombre. Si el nombre se encuentra en la agenda, debe mostrar el teléfono y, opcionalmente, permitir modificarlo si no es correcto. Si el nombre no se encuentra, debe permitir ingresar el teléfono correspondiente.
- Buscar: Nos pide una cadena de caracteres, y nos muestras todos los contactos cuyos nombres comiencen por dicha cadena.
- Borrar: Nos pide un nombre y si existe nos preguntará si queremos borrarlo de la agenda.
- Listar: Nos muestra todos los contactos de la agenda.

In [1]:
agenda = {}

while True:
    print("\n--- MENÚ AGENDA ---")
    print("1. Añadir / Modificar")
    print("2. Buscar")
    print("3. Borrar")
    print("4. Listar")
    print("5. Salir")

    opcion = input("Elige una opción: ")

    if opcion == "1":
        nombre = input("Introduce el nombre: ")
        if nombre in agenda:
            print("El teléfono actual de", nombre, "es:", agenda[nombre])
            cambiar = input("¿Quieres modificarlo? (s/n): ")
            if cambiar.lower() == "s":
                telefono = input("Introduce el nuevo teléfono: ")
                agenda[nombre] = telefono
        else:
            telefono = input("Introduce el teléfono: ")
            agenda[nombre] = telefono

    elif opcion == "2":
        cadena = input("Introduce el comienzo del nombre a buscar: ")
        for nombre in agenda:
            if nombre.startswith(cadena):
                print(nombre, "->", agenda[nombre])

    elif opcion == "3":
        nombre = input("Introduce el nombre a borrar: ")
        if nombre in agenda:
            confirmar = input("¿Seguro que quieres borrarlo? (s/n): ")
            if confirmar.lower() == "s":
                del agenda[nombre]
                print("Contacto eliminado.")
        else:
            print("Ese contacto no existe.")

    elif opcion == "4":
        if len(agenda) == 0:
            print("La agenda está vacía.")
        else:
            for nombre, telefono in agenda.items():
                print(nombre, "->", telefono)

    elif opcion == "5":
        print("Saliendo de la agenda...")
        break

    else:
        print("Opción no válida. Intenta de nuevo.")


--- MENÚ AGENDA ---
1. Añadir / Modificar
2. Buscar
3. Borrar
4. Listar
5. Salir
Saliendo de la agenda...


## Ejercicio 1: agregar estudiantes y sus calificaciones a un diccionario
Crea un programa que permita agregar nombres de estudiantes como claves y sus calificaciones como valores en un diccionario. Luego, imprime el diccionario.

In [2]:
diccionario = {}

bandera = "si"

while bandera == "si":
    nombre = input("Nombre del estudiante: ")
    nota = int(input("Mencione la nota"))
    bandera = input("Deseas continuar si/no?")

diccionario[nombre] = nota

diccionario

{'Randolph': 5}

In [ ]:
# Crear un diccionario vacío
calificaciones = {}

# Pedir el número de estudiantes
num_estudiantes = int(input("¿Cuántos estudiantes quieres agregar?: "))

# Agregar estudiantes y calificaciones al diccionario
for i in range(num_estudiantes):
    nombre = input(f"Introduce el nombre del estudiante {i + 1}: ")
    calificacion = int(input(f"Introduce la calificación de {nombre}: "))
    calificaciones[nombre] = calificacion

# Mostrar el diccionario con las calificaciones
print("Diccionario de estudiantes y calificaciones:", calificaciones)

## Ejercicio 2: buscar un valor en un diccionario
Escribe un programa que permita al usuario buscar una clave en un diccionario de frutas y colores. Si la clave existe, muestra el color; de lo contrario, muestra un mensaje indicando que no se encontró.

In [4]:
diccionario = {"manzana":"rojo", "pera":"verde", "uva":"morada"}

nombre =input("Mecione la fruta: ")

if nombre in diccionario:
    fruta = diccionario[nombre]
    print(fruta)
else:
    print("La fruta no esta en el diccionario")

La fruta no esta en el diccionario


## Ejercicio 3: actualizar los precios de productos en un diccionario
Escribe un programa que permita actualizar el precio de un producto en un diccionario de productos y precios. Si el producto no existe, se debe añadir.

In [2]:
diccionario = {"silla": 500, "mesa":600, "portatil":700}

producto = input("Mencione el producto: ")

if producto in diccionario:
    precio = int(input("Actualice su precio: "))
    diccionario[producto] = precio
else:
    precio = int(input("Introduzca su precio: "))
    diccionario[producto] = precio

diccionario


{'silla': 500, 'mesa': 200, 'portatil': 700}

## Ejercicio 4: eliminar claves con valores negativos en un diccionario
Escribe un programa que elimine las claves de un diccionario que tienen valores negativos. Usa un diccionario con números enteros como ejemplo.

In [7]:
diccionario = {"uno":1, "dos":2, "menos-1":-1, "tres":3, "cuatro":4}

for numero in dict(diccionario):
    valor = diccionario[numero]
    if valor <0:
        del(diccionario[numero])

diccionario

{'uno': 1, 'dos': 2, 'tres': 3, 'cuatro': 4}

In [ ]:
# Definir un diccionario con números enteros
numeros = {
    "a": 5,
    "b": -3,
    "c": 8,
    "d": -1
}

# Crear una lista de claves a eliminar
claves_a_eliminar = [clave for clave, valor in numeros.items() if valor < 0]

# Eliminar las claves con valores negativos
for clave in claves_a_eliminar:
    del numeros[clave]

# Mostrar el diccionario actualizado
print("Diccionario sin valores negativos:", numeros)

## Ejercicio 5: contar la frecuencia de letras en una palabra
Escribe un programa que cuente cuántas veces aparece cada letra en una palabra dada por el usuario, usando un diccionario.

In [8]:
palabra = input("Escriba la palabra")
letra = input("Introduzca la letra que desea mirar")

#busquedad = palabra.startswith(letra)

conteo = 0

for word in palabra:
    if letra == word:
        conteo += 1

print(f"La letra {letra} en la palabra {palabra} aparace {conteo} veces")

La letra a en la palabra araña aparace 3 veces


In [9]:
# Pedir una palabra al usuario
palabra = input("Introduce una palabra: ")

# Crear un diccionario para almacenar la frecuencia de letras
frecuencia_letras = {}

# Contar la frecuencia de cada letra
for letra in palabra:
    if letra in frecuencia_letras:
        frecuencia_letras[letra] += 1
    else:
        frecuencia_letras[letra] = 1

# Mostrar el diccionario con la frecuencia de letras
print("Frecuencia de letras:", frecuencia_letras)

Frecuencia de letras: {'a': 3, 'r': 1, 'ñ': 1}


## Ejercicio 6: intercambiar claves y valores de un diccionario
Escribe un programa que intercambie las claves y valores de un diccionario, es decir, que los valores pasen a ser las claves y las claves pasen a ser los valores.

In [18]:
# Definir un diccionario de ejemplo
diccionario = {
    "nombre": "Juan",
    "edad": 30,
    "ciudad": "Madrid"
}

# Intercambiar las claves y los valores
diccionario_invertido = {valor: clave for clave, valor in diccionario.items()}

# Mostrar el diccionario invertido
print("Diccionario invertido:", diccionario_invertido)

Diccionario invertido: {'Juan': 'nombre', 30: 'edad', 'Madrid': 'ciudad'}


## 🔹 Nivel 1 – Muy Básico (entradas, salidas y operaciones simples)

1. Suma de dos números → Pide dos enteros y muestra la suma.

2. Área de un triángulo → Pide base y altura, calcula el área.

3. Convertidor de grados Celsius a Fahrenheit.

4. Número par o impar → Pide un número y determina si es par o impar.

5. Mayor de dos números → Pide dos números y muestra cuál es mayor.

In [2]:
# 1. Suma de dos números → Pide dos enteros y muestra la suma.

num1 = int(input("Escriba un numero para sumar: "))
num2 = int(input("Escriba el otro numero para sumar: "))

num3 = num1 + num2

print(f"la suma de {num1} y {num2} es de {num3}")

la suma de 3 y 4 es de 7


In [3]:
# 2. Área de un triángulo → Pide base y altura, calcula el área.

a = int(input("Escriba la base del triangulo para calcular el area: "))
b = int(input("Escriba la altura para calcular el area: "))
area = (a*b)/2 
print(f"El area del triangulo es de {area}")

El area del triangulo es de 15.0


In [4]:
# 3. Convertidor de grados Celsius a Fahrenheit.
#(× 9/5) + 32

celsius = float(input("Introduzca los grados celsius: "))

convertidor = (celsius*(9.5)) + 32

print(f"Los grados de {celsius} en celsius en Fahrenheit es de {convertidor}")

Los grados de 54.0 en celsius en Fahrenheit es de 545.0


In [6]:
# 4. Número par o impar → Pide un número y determina si es par o impar.

a = int(input("Introduzca su numero"))

if a % 2 == 0:
    print("Es un numero par")
else:
    print("Es un numero impar")

Es un numero impar


In [ ]:
# 5. Mayor de dos números → Pide dos números y muestra cuál es mayor.

a = float(input("Introduzca el numero: "))
b = float(input("Introduzca el otro numero: "))

if a < b:
    print(f"El numero {b} es mayor que {a}")
elif a > b:
    print(f"El numero {a} es mayor que {b}")
elif a == b:
    print("Los numero son iguales")

El numero 3.0 es mayor que 2.0


## 🔹 Nivel 2 – Manejo de Cadenas

1. Longitud de una cadena → Pide una palabra y muestra cuántas letras tiene.

2. Reverso de una cadena → Pide una palabra y muéstrala al revés.

3. Contar vocales → Pide una frase y cuenta cuántas vocales tiene.

4. Palíndromo → Pide una palabra y verifica si se lee igual al derecho y al revés.

5. Capitalizar palabras → Pide una frase y muestra cada palabra iniciando con mayúscula.

In [2]:
# 1. Longitud de una cadena → Pide una palabra y muestra cuántas letras tiene.

palabra = input("Dame la palabra para contar cuantas letras tiene: ")

contador = len(palabra)

print(f"La palabra {palabra} tiene {contador} letras")

La palabra contador tiene 8 letras


In [3]:
# 2. Reverso de una cadena → Pide una palabra y muéstrala al revés.
palabra = input("Dame la palabra para invertir: ")
palabra_invertida = palabra[::-1]
print(f"la palabra {palabra} invertida es: {palabra_invertida}")

la palabra amor invertida es: roma


In [5]:
# 3. Contar vocales → Pide una frase y cuenta cuántas vocales tiene.

palabra = input("Dame la palabra: ")
vocales = ["a", "e", "i", "o", "u"]
conteo = 0
for vocal in vocales:
    if vocal in palabra:
        conteo +=1

print(f"la palabra {palabra} tiene {conteo} vocales")


la palabra murcielago tiene 5 vocales


In [ ]:
# 4. Palíndromo → Pide una palabra y verifica si se lee igual al derecho y al revés.

palabra = input("Dame la palabra: ")
palabra_invertida = palabra[::-1]

if palabra == palabra_invertida:
    print(f"La palabra {palabra} es un palindromo")
else:
    print(f"La palabra {palabra} no es un palindromo")

La palabra murcielago no es un palindromo


In [9]:
# 5. Capitalizar palabras → Pide una frase y muestra cada palabra iniciando con mayúscula.

palabra = input("Dame la palabra: ")
print(palabra.title())

Randolph Rafael Peralta Monterroza


## 🔹 Nivel 3 – Condicionales y Operaciones

1. Clasificador de edades → Según la edad ingresada, muestra “niño”, “adolescente”, “adulto” o “anciano”.

2. Calculadora simple → Pide dos números y una operación (+, -, *, /).

3. Comparador de tres números → Pide tres números y muestra el mayor.

4. Año bisiesto → Pide un año y determina si es bisiesto.

5. Conversor de unidades → Pide una cantidad en metros y conviértela a cm, mm y km.

In [ ]:
#1. Clasificador de edades → Según la edad ingresada, muestra “niño”, “adolescente”, “adulto” o “anciano”.

edad = int(input("Ingrese su edad: "))

if edad < 0:
    print("Esto no es una edad")
elif edad < 11:
    print("Eres un niño")
elif edad < 18:
    print("Eres un adolescente")
elif edad < 40:
    print("Eres un adulto")
else:
    print("Eres un anciano")    

Eres un anciano


In [ ]:
# 2. Calculadora simple → Pide dos números y una operación (+, -, *, /).
a = float(input("Dame un numero: "))
b = float(input("Dame otro numero"))

suma = a + b
resta = a - b
multiplicacion = a*b
division = a/b

operacion = input("Ingrese la operacion que desea +, -, *, /: ")

if operacion == "suma" or operacion == "+":
    print(suma)
elif operacion == "resta" or operacion == "-":
    print(resta)
elif operacion == "multiplicacion" or operacion == "*":
    print(multiplicacion)
elif operacion == "division" or operacion == "/":
    print(division)
else:
    print("escriba bien la operacion") 


2.3333333333333335


In [ ]:
# 3. Comparador de tres números → Pide tres números y muestra el mayor.

a = float(input("Dame el primer numero: "))
b = float(input("Dame el segundo numero"))
c = float(input("Dame el tercer numero"))

if b<a and c<a:
    print(f"el numero mayor es {a}")
elif a<b and c<b:
    print(f"el numero mayor es {b}")
elif a<c and b<c:
    print(f"el numero mayor es {c}")
else:
    print("No se sabe que numero es mayor")

el numero mayor es 4.0


In [30]:
# 4. Año bisiesto → Pide un año y determina si es bisiesto.

anio = int(input("Ingrese el año: "))

if anio%4 == 0:
    print("Es un año bisiesto")
else:
    print("No es año bisieto")

No es año bisieto


In [32]:
# 5. Conversor de unidades → Pide una cantidad en metros y conviértela a cm, mm y km.

metros = float(input("Ingrese la medida en metros"))
centimetros = metros*100
milimetro = metros*1000
kilometros = metros*(0.1)
print(f"la medida {metros} metros en centimetros: {centimetros}, milimetros: {milimetro}, kilometros: {kilometros}")

la medida 900.0 metros en centimetros: 90000.0, milimetros: 900000.0, kilometros: 90.0


## 🔹 Nivel 4 – Bucles y Construcción de Patrones

1. Tabla de multiplicar → Pide un número y muestra su tabla hasta el 10.

2. Suma de los primeros N números → Pide un número y calcula la suma de 1 hasta N.

3. Factorial de un número → Pide un número y calcula su factorial.

4. Adivina el número → El programa genera un número aleatorio y el usuario debe adivinarlo.

5. Pirámide de asteriscos → Pide un número de filas y dibuja una pirámide.

In [5]:
 #1. Tabla de multiplicar → Pide un número y muestra su tabla hasta el 10.

numero = int(input("Ingrese su numero: "))

for i in range(11):
    print(i*numero)

0
7
14
21
28
35
42
49
56
63
70


In [6]:
#2. Suma de los primeros N números → Pide un número y calcula la suma de 1 hasta N.

numero = int(input("Ingrese su numero: "))

gauss = numero*(numero + 1)/2

print(f"la suma de 1 a {numero} es de {gauss}")

la suma de 1 a 9 es de 45.0


In [ ]:
# 3. Factorial de un número → Pide un número y calcula su factorial.

numero = int(input("Ingrese su numero: "))
uno = 1
for i in range(1, numero+1):
    uno *= i

print(f"El factorial del numero {numero} es {uno}")

El factorial del numero 4 es 24


In [25]:
# 4. Adivina el número → El programa genera un número aleatorio y el usuario debe adivinarlo.

adivinanza = 5

numero = int(input("Adivina el numero: "))

if numero == adivinanza:
    print("Acertaste ese es el numero")
else:
    print("No es el numero")


Acertaste ese es el numero


In [28]:
# 5. Pirámide de asteriscos → Pide un número de filas y dibuja una pirámide.

numero = int(input("Ingrese su numero: "))

for i in range(numero+1):
    print(i*"*")



*
**
***
****
*****
******
*******


## 🔹 Nivel 5 – Ligeramente más complejos (combinando todo)

1. Número primo → Pide un número y verifica si es primo.

2. Serie de Fibonacci → Muestra los primeros N términos de la serie.

3. Inversión de un número → Pide un número entero y muestra su inverso (ej: 123 → 321).

4. Conversor de moneda → Convierte una cantidad en pesos colombianos a dólares y euros.

5. Calculadora de promedio → Pide N notas y muestra el promedio y si aprobó o reprobó.

In [9]:
# 1. Número primo → Pide un número y verifica si es primo.

numero = int(input("Ingrese su numero: "))

for i in range(2, numero+1):
    if numero%i == 0:
        break
if i == numero:
    print(f"El numero {numero} es un numero primo")
else:
    print(f"El numero {numero} no es un numero primo")

El numero 10 no es un numero primo


In [7]:
n = int(input("Introduce un número entero positivo mayor que 2: "))
i = 2
while n % i != 0:
    i += 1
if i == n:
    print(str(n) + " es primo")
else:
    print(str(n) + " no es primo")

7 es primo


In [1]:
# 2. Serie de Fibonacci → Muestra los primeros N términos de la serie.

# Definir el número de términos que queremos generar
num_terminos = int(input("Escriba el numero hasta donde desea la serie de fibonaci: "))

# Inicializar las variables para los dos primeros números de Fibonacci
a = 0
b = 1

# Imprimir los dos primeros términos si num_terminos es mayor que 0 y 1 respectivamente
if num_terminos >= 1:
    print(a)
if num_terminos >= 2:
    print(b)

# Generar el resto de los términos de la secuencia
for i in range(2, num_terminos):
    siguiente_termino = a + b
    print(siguiente_termino)
    # Actualizar los valores para la siguiente iteración
    a = b
    b = siguiente_termino

0
1
1
2
3
5
8
13
21
34


In [2]:
# 3. Inversión de un número → Pide un número entero y muestra su inverso (ej: 123 → 321).

def invertir_numero_con_bucle(numero):
  numero_invertido = 0
  while numero > 0:
    # Obtener el último dígito
    digito = numero % 10
    # Agregar el dígito al número invertido
    numero_invertido = (numero_invertido * 10) + digito
    # Eliminar el último dígito del número original
    numero = numero // 10
  return numero_invertido

# Ejemplo de uso
numero_original = 12345
numero_invertido = invertir_numero_con_bucle(numero_original)
print(f"El número original es: {numero_original}")
print(f"El número invertido es: {numero_invertido}")



El número original es: 12345
El número invertido es: 54321


In [4]:
numero = input("Ingrese su numero: ")
numero_invertido = numero[::-1]
print(f"El numero {numero} invertido se muestra asi: {numero_invertido}")



El numero 567890 invertido se muestra asi: 098765


In [6]:
#4. Conversor de moneda → Convierte una cantidad en pesos colombianos a dólares y euros.

pesos = float(input("Ingrese los pesos colombianos a convertir: "))

dolares = float(input("Ingrese la tasa de cambio actual del dolar"))

euros = float(input("Ingrese la tasa de cambio actual del euro"))

conversion_dolar = pesos/dolares

conversion_euro = pesos/euros

print(f"La cantidad de {pesos} pesos en dolares: {dolares} y euros: {euros}")

La cantidad de 1000.0 pesos en dolares: 3986.15 y euros: 4673.57


In [11]:
# 5. Calculadora de promedio → Pide N notas y muestra el promedio y si aprobó o reprobó.

cantidad = int(input("Ingrese la cantidad de notas que desea: "))

notas = []

for i in range(cantidad):
    nota = float(input("Ingrese la nota: "))
    notas.append(nota)

suma_notas = 0

for j in notas:
    suma_notas += j

promedio = suma_notas/cantidad

print(f"El promedio de notas es de {promedio}")

El promedio de notas es de 3.8


## Nivel 6 – Retadores con tipos simples

1. Número perfecto → Determina si un número es perfecto (suma de sus divisores propios = número).

2. Números amigos → Verifica si dos números son amigos (ej: 220 y 284).

3. Binario a decimal y decimal a binario.

4. Conversor de tiempo → Convierte segundos a horas, minutos y segundos.

5. Simulación de cajero automático → Pide saldo inicial y permite ingresar o retirar dinero, validando que no quede en negativo.

In [ ]:
# 1. Número perfecto → Determina si un número es perfecto (suma de sus divisores propios = número).
numero = int(input("Ingrese su numero perfecto: "))
suma = 0

for i in range(1, numero):
    if numero%i == 0:
        suma += i
if suma == numero:
    print(f"El numero {numero} es un numero perfecto")
else:
    print(f"El numero {numero} no es un numero perfecto")

No es un numero perfecto


In [5]:
#2. Números amigos → Verifica si dos números son amigos (ej: 220 y 284).

numero = int(input("Ingrese su primer numero: "))
#otro_numero = int(input("Ingrese su segundo numero: "))

suma = 0
for i in range(1, numero):
    if numero%i == 0:
        suma += i
suma

otro_numero = int(input("Ingrese su segundo numero: "))
otra_suma = 0
for j in range(1, otro_numero):
    if otro_numero%j == 0:
        otra_suma += j
otra_suma

if suma == otro_numero and otra_suma == numero:
    print(f"El numero {numero} y el numero {otro_numero} son amigos")
else:
    print(f"El numero {numero} y el numero {otro_numero} no son amigos")

El numero 220 y el numero 284 son amigos


In [3]:
#3. Binario a decimal y decimal a binario.

def binario_a_decimal(binario):
    decimal = 0
    potencia = 0
    # Convertir el número binario a cadena para poder recorrerlo
    binario_str = str(binario)
    # Recorrer los dígitos de derecha a izquierda
    for digito in reversed(binario_str):
        # Sumar el dígito (convertido a entero) multiplicado por 2^potencia
        decimal += int(digito) * (2 ** potencia)
        potencia += 1
    return decimal

# Ejemplo de uso
numero_binario = 10110
resultado_decimal = binario_a_decimal(numero_binario)
print(f"El binario {numero_binario} es {resultado_decimal} en decimal") # Salida: El binario 10110 es 22 en decimal

El binario 10110 es 22 en decimal


In [4]:
#3. Binario a decimal y decimal a binario.

def decimal_a_binario(decimal):
    if decimal == 0:
        return "0"

    binario_str = ""
    while decimal > 0:
        # Obtener el residuo de la división entre 2
        residuo = decimal % 2
        # Añadir el residuo (como string) al inicio de la cadena binaria
        binario_str = str(residuo) + binario_str
        # Actualizar el decimal con el cociente de la división
        decimal = decimal // 2
    return binario_str

# Ejemplo de uso
numero_decimal = 22
resultado_binario = decimal_a_binario(numero_decimal)
print(f"El decimal {numero_decimal} es {resultado_binario} en binario") # Salida: El decimal 22 es 10110 en binario

El decimal 22 es 10110 en binario


In [1]:
# 4. Conversor de tiempo → Convierte segundos a horas, minutos y segundos.

segundos = float(input("Ingrese los segundos a convertir"))

minutos = segundos/60

horas = minutos/60

print(f"Los segundos: {segundos}, los minutos son: {minutos}, las horas son: {horas}")

Los segundos: 12345.0, los minutos son: 205.75, las horas son: 3.4291666666666667


In [ ]:
#5. Simulación de cajero automático → Pide saldo inicial y permite ingresar o retirar dinero, validando que no quede en negativo.

opcion = input("Que opcion quieres tomar (a) retirar, (b) ingresar")

if opcion == 'a':
    saldo_inicial = float(input("Ingrese su saldo inicial"))
    ingreso = float(input("Cuanto desea ingresar"))
    saldo_total = saldo_inicial + ingreso
    print(f"Su ingreso total es de {saldo_total}")

elif opcion == 'b':
    saldo_inicial = float(input("Ingrese su saldo inicial"))
    retiro = float(input("Cuanto desea retirar"))
    if saldo_inicial>retiro:
        saldo_resultante = saldo_inicial - retiro
    else:
        print("El saldo es insuficiente")
    print(f"Su ingreso total es de {saldo_resultante}")

else:
    print("La opcion no es la correcta")

Su ingreso total es de 223456.0


## Nivel 1 – Muy básicos (lectura y operaciones simples con strings)

1. Longitud de una cadena → Pide una palabra y muestra cuántos caracteres tiene.

2. Concatenación de cadenas → Pide nombre y apellido, y muéstralos juntos.

3. Repetición de cadenas → Pide una palabra y repítela N veces.

4. Mayúsculas y minúsculas → Pide un texto y muéstralo en mayúsculas, minúsculas y con la primera letra en mayúscula.

5. Primer y último carácter → Pide una palabra y muestra su primer y último carácter.

In [ ]:
# 1. Longitud de una cadena → Pide una palabra y muestra cuántos caracteres tiene.

palabra = input("Ingrese su palabra: ")

print(f"la palabra {palabra} tiene {len(palabra)} caracteres")

['A', 'm', 'o', 'r']

In [3]:
# 2. Concatenación de cadenas → Pide nombre y apellido, y muéstralos juntos.

nombre = input("ingrese su nombre: ")

apellido = input("ingrese su apellido: ")

print("Usted se llama", nombre, apellido)

Usted se llama Randolph Peralta


In [3]:
# 3. Repetición de cadenas → Pide una palabra y repítela N veces.

palabra = input("Dame una palabra: ")

veces = int(input("Cuantas veces deseas que se repita: "))

for i in range(veces):
    print(palabra)


amor
amor
amor
amor
amor
amor
amor
amor
amor
amor


In [3]:
# 4. Mayúsculas y minúsculas → Pide un texto y muéstralo en mayúsculas, minúsculas y con la primera letra en mayúscula.

palabra = input("Escriba la palabra: ")

print(f"La palabra {palabra} en mayuscula: {palabra.upper()}, en minuscula: {palabra.lower()}, en titulo: {palabra.title()}")

La palabra rAndolph en mayuscula: RANDOLPH, en minuscula: randolph, en titulo: Randolph


In [6]:
#5. Primer y último carácter → Pide una palabra y muestra su primer y último carácter.

palabra = input("Escriba la palabra: ")

print(f"La palabra {palabra} tiene como inicial {palabra[0]} y la su letra final {palabra[-1]}")

La palabra Randolph tiene como inicial R y la su letra final h


## Nivel 2 – Indexación y slicing

6. Reverso de una cadena → Muestra la cadena al revés.

7. Subcadena → Pide una palabra y muestra los primeros 3 caracteres y los últimos 3.

8. Extraer iniciales → Pide nombre y apellido, y devuelve sus iniciales (ej: "Randolph Peralta" → "RP").

9. Verificar prefijo/sufijo → Pide una palabra y comprueba si empieza con “a” o termina con “z”.

10. Eliminar espacios → Pide una frase y elimina los espacios en blanco al inicio y final.

In [2]:
# 6. Reverso de una cadena → Muestra la cadena al revés.

string = input("Dame una cadena (palabra) para revertirla: ")

print(f"La palabra o cadena: {string}, al reves seria: {string[::-1]}")

La palabra o cadena: amor, al reves seria: roma


In [ ]:
# 7. Subcadena → Pide una palabra y muestra los primeros 3 caracteres y los últimos 3.

string = input("Dame una cadena (palabra): ")

print(f"la palabra {string} tiene los tres primeros caracteres: {string[:3]}, y sus tres ultimos caracteres son {string[-3:]}")

la palabra helicoptero tiene los tres primeros caracteres: hel, y sus tres ultimos caracteres son ero


In [9]:
# 8. Extraer iniciales → Pide nombre y apellido, y devuelve sus iniciales (ej: "Randolph Peralta" → "RP").

string = input("Dame una cadena (palabra): ")

string_sep = string.split()

initial = ""

for str in string_sep:
    initial += str[0]

print(f"Las iniciales de nombre {string} son: {initial}")

Las iniciales de nombre Randolph Peralta son: RP


In [ ]:
# 9. Verificar prefijo/sufijo → Pide una palabra y comprueba si empieza con “a” o termina con “z”.

palabra = input("Escriba una palabra: ")

if palabra[0] == "a":
    print(f"La palabra {palabra} empieza con a")
elif palabra[-1] == "z":
    print(f"la palabra {palabra} termina con z")
else:
    print(f"La palabra {palabra} no comienza a ni termina en z")

la palabra quiroz termina con z


In [5]:
#10. Eliminar espacios → Pide una frase y elimina los espacios en blanco al inicio y final.

palabra = input("Escriba una palabra: ")

palabra[0].split()
palabra[-1].split()

print(palabra)

hola mundo como estan


## Nivel 3 – Conteo y búsqueda

11. Contar vocales → Pide una frase y cuenta cuántas vocales tiene.

12. Contar palabras → Pide una frase y cuenta cuántas palabras tiene.

13. Buscar subcadena → Pide una frase y una palabra, y verifica si aparece en la frase.

14. Reemplazar caracteres → Pide un texto y reemplaza todas las vocales por “*”.

15. Frecuencia de letra → Pide una frase y una letra, y cuenta cuántas veces aparece.

In [ ]:
#11. Contar vocales → Pide una frase y cuenta cuántas vocales tiene.

frase = input("Escriba una frase o palabra: ")

vocales = ['a', 'e', 'i', 'o', 'u']

conteo = 0

for i in frase:
    if i in vocales:
        conteo +=1

print(f"La frase o palabra {frase} tiene {conteo} vocales")

La frase o palabra amor tiene 2 vocales


In [2]:
# 12. Contar palabras → Pide una frase y cuenta cuántas palabras tiene.

frase = input("Escriba una frase o palabra: ")

frase_sep = frase.split()

conteo = 0

for fr in frase_sep:
    conteo += 1

print(f"la frase o palabra: {frase}, tiene {conteo} palabras")

la frase o palabra: amor es dificil, tiene 3 palabras


In [4]:
# 13. Buscar subcadena → Pide una frase y una palabra, y verifica si aparece en la frase.

frase = input("Escriba una frase: ")

palabra = input("Escriba la palabra: ")

frase_sep = frase.split()

if palabra in frase_sep:
    print(f"La palabra {palabra} esta en la frase {frase}")
else:
    print(f"La palabra {palabra} no esta en la frase {frase}")

La palabra amarillo no esta en la frase el lapiz es de color azul


In [1]:
# 14. Reemplazar caracteres → Pide un texto y reemplaza todas las vocales por “*”.

frase = input("Escriba una frase o palabra: ")

vocales = ['a', 'e', 'i', 'o', 'u']

frase_nueva = ""

for i in frase:
    if i in vocales:
        frase_nueva += "*"
    else:
        frase_nueva += i

print(frase_nueva)

*m*r*ll*


In [9]:
# 15. Frecuencia de letra → Pide una frase y una letra, y cuenta cuántas veces aparece.

frase = input("Escriba una frase o palabra: ")

letra = input("Menciona la letra: ")

conteo = 0

for i in frase:
    if letra == i:
        conteo += 1

print(f"la frase {frase} aparace la letra {letra}: {conteo} veces")

la frase amarillo aparace la letra a: 2 veces


In [6]:
frase = input("Escriba una frase o palabra: ")

diccionario = {}

for i in frase:
    if i in diccionario:
        diccionario[i] += 1
    else:
        diccionario[i] = 1

print(diccionario)

{'a': 2, 'm': 1, 'r': 1, 'i': 1, 'l': 2, 'o': 1}


## Nivel 4 – Condiciones y validaciones

16. Palíndromo → Verifica si una palabra o frase es palíndroma.

17. Verificar solo números → Comprueba si una cadena contiene solo dígitos.

18. Validación de correo → Comprueba si un texto tiene “@” y “.com”.

19. Contraseña segura → Verifica si una contraseña tiene al menos 8 caracteres, una mayúscula, una minúscula y un número.

20. Eliminar duplicados → Pide una frase y elimina caracteres repetidos.

In [ ]:
# 16. Palíndromo → Verifica si una palabra o frase es palíndroma.

frase = input("Escriba una frase o palabra: ")

esarf = frase[::-1]

if frase == esarf:
    print(f"La palabra o frase {frase} es un palindromo")
else:
    print(f"La palabra o frase {frase} no es un palindromo")

La palabra o frase ana es un palindromo


In [10]:
# 17. Verificar solo números → Comprueba si una cadena contiene solo dígitos.

cadena = input("Escriba una cadena o palabra: ")
es_numero = False

digitos = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

frase_espacio = cadena.strip()

for i in digitos:
    if i in frase_espacio:
        es_numero = True
        break

if es_numero:
    print(f"La cadena {cadena} contiene digitos")
else:
    print(f"La cadena {cadena} no contiene digitos")

La cadena Randlph123 contiene digitos


In [3]:
# 18. Validación de correo → Comprueba si un texto tiene “@” y “.com”.

correo = input("Ingresa el correo: ")

if "@" in correo and ".com" in correo:
    print("El correo es válido (contiene '@' y '.com')")
else:
    print("El correo no es válido")

El correo es válido (contiene '@' y '.com')


In [2]:
# 19. Contraseña segura → Verifica si una contraseña tiene al menos 8 caracteres, una mayúscula, una minúscula y un número.

contraseña = input("Ingresa la contraseña: ")

tiene_mayuscula = False
tiene_minuscula = False
tiene_numero = False

# Verificar cada carácter
for caracter in contraseña:
    if caracter >= 'A' and caracter <= 'Z':
        tiene_mayuscula = True
    elif caracter >= 'a' and caracter <= 'z':
        tiene_minuscula = True
    elif caracter >= '0' and caracter <= '9':
        tiene_numero = True

# Verificar longitud y condiciones
if len(contraseña) >= 8 and tiene_mayuscula and tiene_minuscula and tiene_numero:
    print("La contraseña es segura ✅")
else:
    print("La contraseña no cumple con los requisitos ❌")

La contraseña no cumple con los requisitos ❌


In [7]:
# 20. Eliminar duplicados → Pide una frase y elimina caracteres repetidos.

frase = input("Escribe una frase: ")
resultado = ""
# Recorremos cada carácter
for caracter in frase:
    # Si el carácter no está ya en resultado, lo agregamos
    if caracter not in resultado:
        resultado += caracter

print("Frase sin caracteres duplicados:")
print(resultado)

Frase sin caracteres duplicados:
el amordiv


In [ ]:
frase = input("Escribe una frase: ")

d = []
for i in frase.split():
    if i not in d:
        d.append(i)
    frase_sin_dupli = " ".join(d)

frase_sin_dupli

'el amor de mi vida'

## Nivel 5 – Transformaciones más elaboradas

21. Invertir palabras → Pide una frase y muestra cada palabra invertida, pero en el mismo orden.

22. Ordenar palabras alfabéticamente → Pide una frase y muestra las palabras en orden alfabético.

23. Censurar palabras prohibidas → Pide un texto y reemplaza palabras de una lista prohibida por “***”.

24. Convertir a formato título → Pide una frase y muestra cada palabra iniciando en mayúscula.

25. Formato fecha → Convierte una fecha ingresada como "2025-08-20" a "20/08/2025".

In [1]:
# 21. Invertir palabras → Pide una frase y muestra cada palabra invertida, pero en el mismo orden.

palabra = input("Ingrese la palabra: ")
palabra_invertida = palabra[::-1]
print(f"La palabra frase {palabra} a lo inverso seria lo siguiente: {palabra_invertida}")

La palabra frase amor a lo inverso seria lo siguiente: roma


In [ ]:
# 22. Ordenar palabras alfabéticamente → Pide una frase y muestra las palabras en orden alfabético.

frase = input("Escribe una frase: ")
lista_frase = frase.split()
lista_frase.sort()
d = []
for i in lista_frase:
    d.append(i)
    frase_sin = " ".join(d)

frase_sin

'amor de el mi vida'

In [4]:
# 23. Censurar palabras prohibidas → Pide un texto y reemplaza palabras de una lista prohibida por “***”.

prohibidas = ["malo", "tonto", "feo"]

texto = input("Ingrese su frase")

palabra = ""
resultado = ""
i = 0

while i < len(texto):
    caracter = texto[i]

    if caracter != " ":
        palabra += caracter
    else:
        if palabra in prohibidas:
            resultado += "*** "
        else:
            resultado += palabra + " "
        palabra = ""
    i += 1

if palabra != " ":
    if palabra in prohibidas:
        resultado += "*** "
    else:
        resultado += palabra

print(resultado)

eso es *** 


In [4]:
# 25. Formato fecha → Convierte una fecha ingresada como "2025-08-20" a "20/08/2025".
fecha = input("Ingresa la fecha en formato AAAA-MM-DD: ")

# Extraer partes usando índices
año = fecha[0] + fecha[1] + fecha[2] + fecha[3]
mes = fecha[5] + fecha[6]
día = fecha[8] + fecha[9]

# Reorganizar en formato DD/MM/AAAA
nueva_fecha = día + "/" + mes + "/" + año

print("Fecha convertida:", nueva_fecha)

Fecha convertida: 20/08/2025


In [3]:
from datetime import datetime

fecha_original = input("ingrese la fecha: ")
# Convertir la cadena a un objeto datetime
fecha_objeto = datetime.strptime(fecha_original, "%Y-%m-%d")

# Formatear la fecha al formato "DD/MM/YYYY"
fecha_formateada = fecha_objeto.strftime("%d/%m/%Y")

print(fecha_formateada)

20/08/2025


## Nivel 6 – Retadores

26. Codificador César → Aplica un desplazamiento de +3 a cada letra de una palabra.

27. Contar letras y números → Pide un texto y muestra cuántos dígitos y cuántas letras tiene.

28. Contar sílabas (aproximado) → Pide una palabra y cuenta las vocales agrupadas.

29. Texto espejo → Pide una frase y muéstrala como si se viera en un espejo.

30. Juego de anagramas → Verifica si dos palabras son anagramas (ej: "roma" y "amor").

In [ ]:
# 26. Codificador César → Aplica un desplazamiento de +3 a cada letra de una palabra.

palabra = input("Escribe una palabra: ")
resultado = ""
i = 0

while i < len(palabra):
    letra = palabra[i]

    # Solo desplazamos letras minúsculas
    if letra >= 'a' and letra <= 'z':
        nueva_letra = chr((ord(letra) - ord('a') + 3) % 26 + ord('a'))
        resultado += nueva_letra

    # También desplazamos letras mayúsculas
    elif letra >= 'A' and letra <= 'Z':
        nueva_letra = chr((ord(letra) - ord('A') + 3) % 26 + ord('A'))
        resultado += nueva_letra

    else:
        # Si no es letra, la dejamos igual
        resultado += letra

    i += 1

print("Palabra con desplazamiento +3:")
print(resultado)

Palabra con desplazamiento +3:
dohoxbd


In [7]:
# 27. Contar letras y números → Pide un texto y muestra cuántos dígitos y cuántas letras tiene.

palabra = input("ingrese su texto: ")

conteo_letra = 0
conteo_digito = 0
i = 0

while i < len(palabra):
    letra = palabra[i]

    # Solo desplazamos letras minúsculas
    if letra >= 'a' and letra <= 'z':
        conteo_letra += 1

    # También desplazamos letras mayúsculas
    elif letra >= '0' and letra <= '9':
        conteo_digito += 1

    i += 1

print(f"la palabra {palabra} tiene {conteo_letra} letras y {conteo_digito} digitos")

la palabra kjhgfd23456 tiene 6 letras y 5 digitos


In [2]:
# 28. Contar sílabas (aproximado) → Pide una palabra y cuenta las vocales agrupadas.

palabra = input("Escribe una palabra: ")

vocales = "aeiouAEIOU"
i = 0
contador = 0
en_vocal = False

while i < len(palabra):
    letra = palabra[i]

    if letra in vocales:
        if not en_vocal:
            contador += 1
            en_vocal = True
    else:
        en_vocal = False

    i += 1

print("Cantidad aproximada de sílabas:", contador)

Cantidad aproximada de sílabas: 2


In [1]:
#29. Texto espejo → Pide una frase y muéstrala como si se viera en un espejo.

frase = input("Escribe una frase: ")

resultado = ""
i = len(frase) - 1

# Recorremos la frase desde el final hacia el inicio
while i >= 0:
    resultado += frase[i]
    i -= 1

print("Texto espejo:")
print(resultado)

Texto espejo:
oiradeceba


In [2]:
#30. Juego de anagramas → Verifica si dos palabras son anagramas (ej: "roma" y "amor").

palabra1 = input("Escribe la primera palabra: ")
palabra2 = input("Escribe la segunda palabra: ")

# Verificamos que tengan la misma longitud
if len(palabra1) != len(palabra2):
    print("No son anagramas ❌")
else:
    i = 0
    es_anagrama = True

    while i < len(palabra1):
        letra = palabra1[i]
        contador1 = 0
        contador2 = 0

        # Contamos cuántas veces aparece la letra en cada palabra
        j = 0
        while j < len(palabra1):
            if palabra1[j] == letra:
                contador1 += 1
            j += 1

        j = 0
        while j < len(palabra2):
            if palabra2[j] == letra:
                contador2 += 1
            j += 1

        if contador1 != contador2:
            es_anagrama = False
            break

        i += 1

    if es_anagrama:
        print("¡Son anagramas! ✅")
    else:
        print("No son anagramas ❌")

¡Son anagramas! ✅


# CONDICIONALES
## Nivel 1 – Básicos (if, else)

1. Número positivo o negativo → Pide un número e indica si es positivo, negativo o cero.

2. Par o impar → Verifica si un número es par o impar.

3. Mayor de dos números → Pide dos números e indica cuál es mayor o si son iguales.

4. Aprobado o reprobado → Pide una nota (0–100) e indica si aprobó (≥ 60).

5. Edad mínima para votar → Pide una edad y verifica si puede votar (≥ 18).

In [8]:
# 1. Número positivo o negativo → Pide un número e indica si es positivo, negativo o cero.
numero = int(input("Ingrese su numero"))

if numero < 0:
    print(f"el numero {numero} es negativo")
elif numero > 0:
    print(f"el numero {numero} es positivo")
elif numero == 0:
    print(f"El numero {numero} es cero")

el numero 8765 es positivo


In [11]:
# 2. Par o impar → Verifica si un número es par o impar.
numero = int(input("Ingrese su numero"))

if numero % 2 == 0:
    print(f"El numero {numero} es par")
else:
    print(f"El numero {numero} es impar")

El numero 23 es impar


In [12]:
# 3. Mayor de dos números → Pide dos números e indica cuál es mayor o si son iguales.
numero1 = int(input("Ingrese su primer numero"))
numero2 = int(input("Ingrese su primer numero"))

if numero1 < numero2:
    print(f"El numero {numero1} es menor que {numero2}")
elif numero2 < numero1:
    print(f"El numero {numero1} es mayor que el numero {numero2}")
elif numero1 == numero2:
    print("Los numeros son iguales")

El numero 2 es mayor que el numero 1


In [13]:
# 4. Aprobado o reprobado → Pide una nota (0–100) e indica si aprobó (≥ 60).
nota = int(input("Ingrese su nota: "))

if nota >= 60:
    print("Aprobo")
else:
    print("No aprobo")

Aprobo


In [14]:
# 5. Edad mínima para votar → Pide una edad y verifica si puede votar (≥ 18).
edad= int(input("Ingrese su edad: "))

if edad >= 18:
    print("Eres mayor de edad")
else:
    print("Eres menor de edad")

Eres mayor de edad


## Nivel 2 – Comparaciones múltiples (if, elif, else)

6. Clasificación de edades → Niño (0–12), Adolescente (13–17), Adulto (18–64), Anciano (65+).

7. Mayor de tres números → Pide tres números y muestra el mayor.

8. Conversor de calificaciones → Convierte una nota (0–100) en letra (A, B, C, D, F).

9. Número divisible → Pide un número e indica si es divisible entre 2, 3 o 5.

10. Calculadora simple → Pide dos números y una operación (+, -, *, /).

In [ ]:
#6. Clasificación de edades → Niño (0–12), Adolescente (13–17), Adulto (18–64), Anciano (65+).
edad= int(input("Ingrese su edad: "))

if edad < 13:
    print("Eres un niño")
elif edad <18:
    print("Eres un adolescente")
elif edad < 64:
    print("Eres un adulto")
else:
    print("Eres un anciano")

Eres un adulto


In [ ]:
#7. Mayor de tres números → Pide tres números y muestra el mayor.

numero1 = int(input("Ingrese el primer numero: "))
numero2 = int(input("Ingrese el segundo numero: "))
numero3 = int(input("Ingrese el tercer numero: "))

if numero1<numero3 and numero2<numero3:
    print(f"el tercer numero: {numero3} es mayor")
elif numero1<numero2 and numero3<numero2:
    print(f"el segundo numero: {numero2} es mayor")
elif numero3<numero1 and numero2<numero1:
    print(f"el primner numero: {numero1} es mayor")

el numero 55 es mayor


In [12]:
# 8. Conversor de calificaciones → Convierte una nota (0–100) en letra (A, B, C, D, F).

nota = int(input("Ingrese su nota: "))

if 90<nota and nota<=100:
    print("A")
elif 80 <nota and nota<90:
    print("B")
elif 70<nota and nota<80:
    print("C")
elif 60<nota and nota<70:
    print("D")
elif 50<nota and nota<60:
    print("E")
elif nota<50:
    print("F")

A


In [14]:
# 9. Número divisible → Pide un número e indica si es divisible entre 2, 3 o 5.

numero = int(input("Ingrese su numero: "))

if numero % 2==0 and numero%3 ==0 and numero%5==0:
    print(f"el numero {numero} es divisible por 2, 3 y 5")
else:
    print(f"El numero {numero} no es divisible por 2,3 y 5")

El numero 31 no es divisible por 2,3 y 5


In [18]:
# 10. Calculadora simple → Pide dos números y una operación (+, -, *, /).

a = float(input("Ingrese su primer numero: "))
b = float(input("Ingrese su segundo numero: "))

operacion = input("Ingrese su opeacion: +, -, *, /")

if operacion == "+":
    print(a + b)
elif operacion == "-":
    print(a -b)
elif operacion == "*":
    print(a*b)
elif operacion == "/":
    print(a/b)

2.0


## Nivel 3 – Condiciones anidadas

11. Triángulo válido → Verifica si tres lados forman un triángulo válido.

12. Tipo de triángulo → Con tres lados válidos, determina si es equilátero, isósceles o escaleno.

13. Número dentro de rango → Verifica si un número está entre 1 y 100.

14. Año bisiesto → Verifica si un año es bisiesto.

15. Comparador de caracteres → Pide una letra e indica si es vocal, consonante, número o símbolo.

In [ ]:
# 11. Triángulo válido → Verifica si tres lados forman un triángulo válido.
lado1 = float(input("Ingrese su lado: "))
lado2 = float(input("Ingrese su segundo lado: "))
lado3 = float(input("Ingrese su tercer lado: "))

if lado1>0 and lado2>0 and lado3>0:
    print("El triangulo es valido")
else:
    print("El triangulo no es valido")

El triangulo es valido


In [5]:
# 12. Tipo de triángulo → Con tres lados válidos, determina si es equilátero, isósceles o escaleno.

lado1 = float(input("Ingrese su lado: "))
lado2 = float(input("Ingrese su segundo lado: "))
lado3 = float(input("Ingrese su tercer lado: "))

if lado1 == lado2 == lado3:
    print("El triangulo es equilatero")
elif lado1 == lado2 or lado2==lado3 or lado3 ==lado1:
    print("El triangulo es isósceles")
else:
    print("El triangulo es escaleno")

El triangulo es isósceles


In [7]:
#13. Número dentro de rango → Verifica si un número está entre 1 y 100. 

numero = int(input("Ingrese su numero: "))

if numero in range(1,100):
    print("El numero esta en el rango")
else:
    print("El numero no esta en el rango")

El numero no esta en el rango


In [12]:
# 14. Año bisiesto → Verifica si un año es bisiesto.

año = int(input("Ingrese su año: "))

if año % 4 == 0 and not año % 100 == 0:
    print("Es un año bisciesto")
else:
    print("No es un año bisciesto")

Es un año bisciesto


In [17]:
#15. Comparador de caracteres → Pide una letra e indica si es vocal, consonante, número o símbolo.

caracter = input("Ingrese su caracter")

vocales = ["a", "e", "i", "o", "u"]

if caracter in vocales:
    print("Es una vocal")
elif caracter>= "a" and caracter<="z":
    if caracter not in vocales:
        print("Es una consonante")
elif type(caracter) == int or type(caracter)==float:
    print("Es un numero")
else:
    print("Es un simbolo")

Es un simbolo
